# <span style="font-family: 'Times New Roman', cursive, sans-serif; font-weight: bold; color: brown;font-size: 34px;">Importing Modules</span>


In [ ]:
import numpy as np
import scipy
from scipy import integrate
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib
import scipy.optimize as spo
import csv
from optimparallel import minimize_parallel
import time
from scipy.interpolate import Rbf
from joblib import Parallel, delayed
import multiprocessing
from scipy.optimize import minimize
from scipy.integrate import trapz
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn import linear_model
from scipy.stats import chi2
from scipy.stats import norm
from matplotlib.patches import Ellipse
import matplotlib.colors as mcolors
import os

# <span style="font-family: 'Times New Roman', cursive, sans-serif; font-weight: bold; color: red;font-size: 48px;">Simulations</span>

# <span style="font-family: 'Verdana', cursive, sans-serif; font-weight: bold; color: green;font-size: 38px;">For the Power law PSD Model</span>

# <span style="font-family: 'Palatino', cursive, sans-serif; font-weight: bold; color: blue;font-size: 30px;">Simple Case</span>

In [ ]:
start = time.time()


n_realizations = 100 # number of realizations to simulate
results_list = []

for i in range(n_realizations):

    
    np.random.seed(i)
    
    gamma=2
    alpha1=1.

    N=600 # number of points in simulated light curve
    step=1 # time sampling of simulated light curve

    f_dc=0.1

#noise level

    noise_level=0.03

    sigma_F=f_dc*noise_level

#Fourier frequencies

    w=2*(np.pi)*np.linspace(1,N,N)/N/step

#Power at each frequency depending on power spectrum model. 


    def power (gamma):
    
        p= w**(-gamma)
             
        return p


#Generate two sets of normally distributed random numbers 
    
    aa = np.random.randn(len(w))
    bb = np.random.randn(len(w))
    
# Light curve in frequency domain  = SQRT(p/2) * (a + b*i)

    f_w = np.sqrt(0.5*power(gamma))*(aa + bb*1j)

    f_w[0]=f_dc * len(w)**0.5

#generating noise

    n=np.ones(len(w))*(sigma_F**2)

    n_w=np.sqrt(0.5*n)*(aa + bb*1.j)

    n_w[0] = np.random.randn(1)*sigma_F

# Generating the total flux (Signal + noise)

    f1_w = alpha1 * f_w
    phi_w = f1_w 
    F_w = phi_w+n_w

    f_t = np.real(np.fft.ifft(f_w[:], norm='ortho'))

    Ft = np.real(np.fft.ifft(F_w[:], norm='ortho'))


    def Sigma_phi(alpha1, gamma):
        Sigma_phi_ = (alpha1**2) * power(gamma)
        return Sigma_phi_


    
    def cov_mat(alpha1, gamma):
       dt = np.linspace(0, N, N)
       cov = Sigma_phi(alpha1, gamma)
    
    # calculate sigma for all dt values
       sigma = 2 * np.trapz(cov * np.cos(w * dt[:, np.newaxis]), w, axis=1)
    
    # add noise to the first element of sigma
       sigma[0] += sigma_F**2
    
    # create the covariance matrix by subtracting the absolute difference between indices
       ID = np.arange(sigma.size)
       sigma = sigma[np.abs(ID - ID[:, None])]
    
       return sigma


    
    def log_like_F (alpha1,gamma):
    
        mu=np.mean(Ft)
      
        data=Ft-mu
    
        inv=np.linalg.inv(cov_mat(alpha1,gamma))
    
        sign,logdet=np.linalg.slogdet(2*np.pi*cov_mat( alpha1, gamma))
    
        lh=-0.5*( logdet + np.transpose(data) @ inv @ data )
    
        return lh



    def mnz(args):
        alpha1, gamma = args
        return -log_like_F(alpha1, gamma) + log_like_F(1, 1)




    g1 = np.arange(1.5, 2.5, 0.025)

    diffe = np.zeros(len(g1))
    amp = np.zeros(len(g1))

    results = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.2, g1_), bounds=((0.1, 0.4), (g1_, g1_)))
        for g1_ in g1
    )

    for j, res1 in enumerate(results):
        amp[j] = res1.x[0]
        diffe[j] = -log_like_F(amp[j], g1[j]) + log_like_F(1, 1)

    results_list.append({'amp': amp, 'diffe': diffe})

    rows=zip(g1,amp,diffe)

    with open ('pl_loop.dat', 'a') as f:
        writer=csv.writer(f, delimiter=' ')
        for i in rows:
            writer.writerow(i)

end = time.time()
print(end - start)

# <span style="font-family: 'Palatino', cursive, sans-serif; font-weight: bold; color: blue;font-size: 30px;">LC analysis in Segments</span>

In [ ]:
start = time.time()


n_realizations = 100
results_list = []

for i in range(n_realizations):

    
    np.random.seed(i)
    
    gamma=2
    alpha1=1.

    N=600 # number of points in simulated light curve
    step=1 # time sampling of simulated light curve

    f_dc=0.1

#noise term

    noise_level=0.03

    sigma_F=f_dc*noise_level

#Fourier frequencies

    w=2*(np.pi)*np.linspace(1,N,N)/N/step

#Power at each frequency depending on power spectrum model. 


    def power (gamma):
    
        p= w**(-gamma)
             
        return p


#Generate two sets of normally distributed random numbers 
    
    aa = np.random.randn(len(w))
    bb = np.random.randn(len(w))
    
# Light curve in frequency domain  = SQRT(p/2) * (a + b*i)

    f_w = np.sqrt(0.5*power(gamma))*(aa + bb*1j)

    f_w[0]=f_dc * len(w)**0.5

#generating noise

    n=np.ones(len(w))*(sigma_F**2)

    n_w=np.sqrt(0.5*n)*(aa + bb*1.j)

    n_w[0] = np.random.randn(1)*sigma_F

# Generating the total flux (Signal + noise)

    f1_w = alpha1 * f_w
    phi_w = f1_w 
    F_w = phi_w+n_w

    f_t = np.real(np.fft.ifft(f_w[:], norm='ortho'))

    F_t = np.real(np.fft.ifft(F_w[:], norm='ortho'))

    lim=200
    low=0
    high=1

    Ft=F_t[low*lim:high*lim]


    def Sigma_phi(alpha1, gamma):
        Sigma_phi_ = (alpha1**2) * power(gamma)
        return Sigma_phi_
  
    
    def cov_mat(alpha1, gamma):
       
        dt=np.linspace(0,lim,lim)
        
        cov=Sigma_phi(alpha1, gamma)
    
    # calculate sigma for all dt values
        sigma = 2 * np.trapz(cov * np.cos(w * dt[:, np.newaxis]), w, axis=1)
    
    # add noise to the first element of sigma
        sigma[0] += sigma_F**2
    
    # create the covariance matrix by subtracting the absolute difference between indices
        ID = np.arange(sigma.size)
        sigma = sigma[np.abs(ID - ID[:, None])]
    
        return sigma

    
    def log_like_F (alpha1,gamma):
    
        mu=np.mean(Ft)
      
        data=Ft-mu
    
        inv=np.linalg.inv(cov_mat(alpha1,gamma))
    
        sign,logdet=np.linalg.slogdet(2*np.pi*cov_mat( alpha1, gamma))
    
        lh=-0.5*( logdet + np.transpose(data) @ inv @ data )
    
        return lh


    def mnz(args):
        alpha1, gamma = args
        return -log_like_F(alpha1, gamma) + log_like_F(1, 1)


    g1 = np.arange(1.5, 2.5, 0.025)

    diffe = np.zeros(len(g1))
    amp = np.zeros(len(g1))

    results = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.2, g1_), bounds=((0.1, 0.4), (g1_, g1_)))
        for g1_ in g1
    )

    for j, res1 in enumerate(results):
        amp[j] = res1.x[0]
        diffe[j] = -log_like_F(amp[j], g1[j]) + log_like_F(1, 1)

    results_list.append({'amp': amp, 'diffe': diffe})

    rows=zip(g1,diffe)

    with open ('1.dat', 'a') as f:
        writer=csv.writer(f, delimiter=' ')
        for i in rows:
            writer.writerow(i)

end = time.time()
print(end - start)

# <span style="font-family: 'Palatino', cursive, sans-serif; font-weight: bold; color: blue;font-size: 30px;">Effect of Gaps and Interpolation Schemes</span>

In [ ]:
start = time.time()


n_realizations = 100
results_list = []

for i in range(n_realizations):

    
    np.random.seed(i)
    
    gamma=2
    alpha1=1.

    N=600 # number of points in simulated light curve
    step=1 # time sampling of simulated light curve

    f_dc=0.1

#noise term

    noise_level=0.03

    sigma_F=f_dc*noise_level

#Fourier frequencies

    w=2*(np.pi)*np.linspace(1,N,N)/N/step

#Power at each frequency depending on power spectrum model. 


    def power (gamma):
    
        p= w**(-gamma)
             
        return p


#Generate two sets of normally distributed random numbers 
    
    aa = np.random.randn(len(w))
    bb = np.random.randn(len(w))
    
# Light curve in frequency domain  = SQRT(p/2) * (a + b*i)

    f_w = np.sqrt(0.5*power(gamma))*(aa + bb*1j)

    f_w[0]=f_dc * len(w)**0.5

#generating noise

    n=np.ones(len(w))*(sigma_F**2)

    n_w=np.sqrt(0.5*n)*(aa + bb*1.j)

    n_w[0] = np.random.randn(1)*sigma_F

# Generating the total flux (Signal + noise)

    f1_w = alpha1 * f_w
    phi_w = f1_w 
    F_w = phi_w+n_w

    f_t = np.real(np.fft.ifft(f_w[:], norm='ortho'))

    F_t = np.real(np.fft.ifft(F_w[:], norm='ortho'))

# section to remove points and interpolation

    T=np.linspace(0,N,N)

    # Calculate the number of points to remove
   
    num_points = int(len(T) * 0.1)

    # Randomly select indices to remove with gaps ranging from 1 to 10 points
    remove_indices = []
    num_removed = 0
    while num_removed < num_points:
        index = np.random.randint(0, len(T) - 10)
        gap = np.random.randint(1, 11)
        remove_indices.extend(range(index, min(index + gap, len(T))))
        num_removed += gap

    # Remove the selected data points from the time series and the flux array
    t4 = np.delete(T, remove_indices)
    Ft4 = np.delete(F_t, remove_indices)
    
    

#    interpolation schemes

 #   f = interpolate.interp1d(t4,Ft4,kind='linear',fill_value='extrapolate')
 #    f = Rbf(t4,Ft4,function='cubic',fill_value=1e+10)
    # Ft=f(T)
 
#  This section is for the Gaussian Process Interpolation 
 
# Define the Kriging model with Matern kernel
   
    kernel = Matern(length_scale=0.2, nu=2.5)
    model = GaussianProcessRegressor(kernel=kernel)
    
    # Train the model
    
    model.fit(t4.reshape(-1, 1), Ft4)
    Ft=model.predict(T.reshape(-1, 1))


    def Sigma_phi(alpha1, gamma):
        Sigma_phi_ = (alpha1**2) * power(gamma)
        return Sigma_phi_




    def cov_mat(alpha1, gamma):
        dt = np.linspace(0, N, N)
        cov = Sigma_phi(alpha1, gamma)

# calculate sigma for all dt values
        sigma = 2 * np.trapz(cov * np.cos(w * dt[:, np.newaxis]), w, axis=1)

# add noise to the first element of sigma
        sigma[0] += sigma_F**2

# create the covariance matrix by subtracting the absolute difference between indices
        ID = np.arange(sigma.size)
        sigma = sigma[np.abs(ID - ID[:, None])]

        return sigma

    
    def log_like_F (alpha1,gamma):
    
        mu=np.mean(Ft)
      
        data=Ft-mu
    
        inv=np.linalg.inv(cov_mat(alpha1,gamma))
    
        sign,logdet=np.linalg.slogdet(2*np.pi*cov_mat( alpha1, gamma))
    
        lh=-0.5*( logdet + np.transpose(data) @ inv @ data )
    
        return lh


    def mnz(args):
        alpha1, gamma = args
        return -log_like_F(alpha1, gamma) + log_like_F(1, 1)


    g1 = np.arange(1.5, 2.5, 0.025)

    diffe = np.zeros(len(g1))
    amp = np.zeros(len(g1))

    results = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.2, g1_), bounds=((0.1, 0.4), (g1_, g1_)))
        for g1_ in g1
    )

    for j, res1 in enumerate(results):
        amp[j] = res1.x[0]
        diffe[j] = -log_like_F(amp[j], g1[j]) + log_like_F(1, 1)

    results_list.append({'amp': amp, 'diffe': diffe})

    rows=zip(g1,diffe)

 #   with open ('pl_lin.dat', 'a') as f:
 #   with open ('pl_cub.dat', 'a') as f:
    with open ('pl_gas.dat', 'a') as f:
        writer=csv.writer(f, delimiter=' ')
        for i in rows:
            writer.writerow(i)

end = time.time()
print(end - start)


# <span style="font-family: 'Palatino', cursive, sans-serif; font-weight: bold; color: blue;font-size: 30px;">Effect of Incorrect Noise Modeling</span>

In [ ]:
start = time.time()


n_realizations = 100
results_list = []

for i in range(n_realizations):

    
    np.random.seed(i)
    
    gamma=2
    alpha1=1.

    N=600 # number of points in simulated light curve
    step=1 # time sampling of simulated light curve

    f_dc=0.1

#noise term

    noise_level=0.03

    sigma_F=f_dc*noise_level

#Fourier frequencies

    w=2*(np.pi)*np.linspace(1,N,N)/N/step

#Power at each frequency depending on power spectrum model. 


    def power (gamma):
    
        p= w**(-gamma)
             
        return p


#Generate two sets of normally distributed random numbers 
    
    aa = np.random.randn(len(w))
    bb = np.random.randn(len(w))
    
# Light curve in frequency domain  = SQRT(p/2) * (a + b*i)

    f_w = np.sqrt(0.5*power(gamma))*(aa + bb*1j)

    f_w[0]=f_dc * len(w)**0.5

#generating noise

    n=np.ones(len(w))*(sigma_F**2)

    n_w=np.sqrt(0.5*n)*(aa + bb*1.j)

    n_w[0] = np.random.randn(1)*sigma_F

# Generating the total flux (Signal + noise)

    f1_w = alpha1 * f_w
    phi_w = f1_w 
    F_w = phi_w+n_w

    f_t = np.real(np.fft.ifft(f_w[:], norm='ortho'))

    Ft = np.real(np.fft.ifft(F_w[:], norm='ortho'))


    def Sigma_phi(alpha1, gamma):
        Sigma_phi_ = (alpha1**2) * power(gamma)
        return Sigma_phi_


    
    def cov_mat(alpha1, gamma,sigma_F):
       dt = np.linspace(0, N, N)
       cov = Sigma_phi(alpha1, gamma)
    
    # calculate sigma for all dt values
       sigma = 2 * np.trapz(cov * np.cos(w * dt[:, np.newaxis]), w, axis=1)
    
    # add noise to the first element of sigma
       sigma[0] += sigma_F**2
    
    # create the covariance matrix by subtracting the absolute difference between indices
       ID = np.arange(sigma.size)
       sigma = sigma[np.abs(ID - ID[:, None])]
    
       return sigma


    
    def log_like_F (alpha1,gamma,sigma_F):
    
        mu=np.mean(Ft)
      
        data=Ft-mu
    
        inv=np.linalg.inv(cov_mat(alpha1,gamma,sigma_F))
    
        sign,logdet=np.linalg.slogdet(2*np.pi*cov_mat( alpha1, gamma, sigma_F))
    
        lh=-0.5*( logdet + np.transpose(data) @ inv @ data )
    
        return lh

    def mnz(args):
        alpha1, gamma, sigma_F= args
        return -log_like_F(alpha1, gamma, sigma_F) + log_like_F(1, 1, sigma_F)


    g1 = np.arange(1.5, 2.5, 0.025)

    # Noise level from 0.5 to 2.0 in 10 log steps
    
    ss1, ss2, ss3, ss4, ss5, ss6, ss7, ss8, ss9, ss10 = np.logspace(0.5,2.0,num=10) * sigma_F
    diffe1, diffe2, diffe3, diffe4, diffe5, diffe6, 
    diffe7, diffe8, diffe9, diffe10  = [np.zeros(len(g1)) for i in range(10)]
    amp1, amp2, amp3, amp4, amp5, amp6, amp7, amp8, amp9, amp10 = [np.zeros(len(g1)) for i in range(10)]
  
    
  
    results1 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.2, g1_,ss1), bounds=((0.1, 10), (g1_, g1_),(ss1,ss1)))
        for g1_ in g1
    )

    for j, res1 in enumerate(results1):
        amp1[j] = res1.x[0]
        diffe1[j] = -log_like_F(amp1[j], g1[j],ss1) + log_like_F(1, 1,ss1)
        
        
    results2 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.2, g1_,ss2), bounds=((0.01, 10), (g1_, g1_),(ss2,ss2)))
        for g1_ in g1
    )

    for j, res2 in enumerate(results2):
        amp2[j] = res2.x[0]
        diffe2[j] = -log_like_F(amp2[j], g1[j],ss2) + log_like_F(1, 1,ss2)
        
  
    results3 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.2, g1_,ss3), bounds=((0.01, 10), (g1_, g1_),(ss3,ss3)))
        for g1_ in g1
    )

    for j, res3 in enumerate(results3):
        amp3[j] = res3.x[0]
        diffe3[j] = -log_like_F(amp3[j], g1[j],ss3) + log_like_F(1, 1,ss3)
        
        
    results4 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.2, g1_,ss4), bounds=((0.01, 10), (g1_, g1_),(ss4,ss4)))
        for g1_ in g1
    )

    for j, res4 in enumerate(results4):
        amp4[j] = res4.x[0]
        diffe4[j] = -log_like_F(amp4[j], g1[j],ss4) + log_like_F(1, 1,ss4)
        
        
    results5 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.2, g1_,ss5), bounds=((0.01, 10), (g1_, g1_),(ss5,ss5)))
        for g1_ in g1
    )

    for j, res5 in enumerate(results5):
        amp5[j] = res5.x[0]
        diffe5[j] = -log_like_F(amp5[j], g1[j],ss5) + log_like_F(1, 1,ss5)
        
        
    results6 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.2, g1_,ss6), bounds=((0.01, 10), (g1_, g1_),(ss6,ss6)))
        for g1_ in g1
    )

    for j, res6 in enumerate(results6):
        amp6[j] = res6.x[0]
        diffe6[j] = -log_like_F(amp6[j], g1[j],ss6) + log_like_F(1, 1,ss6)
        
  
    results7 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.2, g1_,ss7), bounds=((0.01, 10), (g1_, g1_),(ss7,ss7)))
        for g1_ in g1
    )

    for j, res7 in enumerate(results7):
        amp7[j] = res7.x[0]
        diffe7[j] = -log_like_F(amp7[j], g1[j],ss7) + log_like_F(1, 1,ss7)
        
        
    results8 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.2, g1_,ss8), bounds=((0.01, 10), (g1_, g1_),(ss8,ss8)))
        for g1_ in g1
    )

    for j, res8 in enumerate(results8):
        amp8[j] = res8.x[0]
        diffe8[j] = -log_like_F(amp8[j], g1[j],ss8) + log_like_F(1, 1,ss8)
        
        
    results9 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.2, g1_,ss8), bounds=((0.01, 10), (g1_, g1_),(ss9,ss9)))
        for g1_ in g1
    )

    for j, res9 in enumerate(results9):
        amp9[j] = res9.x[0]
        diffe9[j] = -log_like_F(amp9[j], g1[j],ss9) + log_like_F(1, 1,ss9)
        
        
    results10 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.2, g1_,ss10), bounds=((0.01, 10), (g1_, g1_),(ss10,ss10)))
        for g1_ in g1
    )

    for j, res10 in enumerate(results10):
        amp10[j] = res10.x[0]
        diffe10[j] = -log_like_F(amp10[j], g1[j],ss10) + log_like_F(1, 1,ss10)
        
        
        
    rows=zip(np.round(g1,2),np.round(diffe1,2),np.round(diffe2,2),np.round(diffe3,2),np.round(diffe4,2),
             np.round(diffe5,2),np.round(diffe6,2),np.round(diffe7,2),np.round(diffe8,2),
             np.round(diffe9,2),np.round(diffe10,2))


    with open ('pl_log_noise.dat', 'a') as f:
        writer=csv.writer(f, delimiter=' ')
        for i in rows:
            writer.writerow(i)

end = time.time()
print((end - start)/3600)

# <span style="font-family: 'Century Gothic', cursive, sans-serif; font-weight: bold; color: purple;font-size: 30px;">Template for Plots</span>

In [ ]:
#___________ SECTIONS __________________________

matplotlib.rc('xtick', labelsize=25) 
matplotlib.rc('ytick', labelsize=25) 
plt.rc('font', size=25)

df = 2
s = -0.5 * chi2.ppf(0.68, df)
s2 = -0.5 * chi2.ppf(0.95, df)
s3 = -0.5 * chi2.ppf(0.997, df)


slope=np.loadtxt('1.dat')[:,0]
l1=np.loadtxt('1.dat')[:,1]
l2=np.loadtxt('2.dat')[:,1]
l3=np.loadtxt('3.dat')[:,1]



slp = np.arange(1.5, 2.5, 0.025)

lf1 = np.zeros(len(slp))
lf2 = np.zeros(len(slp))
lf3 = np.zeros(len(slp))


for i, sl_ in enumerate(slp):
    lf1[i] = np.mean(l1[slope == sl_])
    lf2[i] = np.mean(l2[slope == sl_])
    lf3[i] = np.mean(l3[slope == sl_])



sig1=lf1[lf1==np.min(lf1)]-s
sig2=lf1[lf1==np.min(lf1)]-s2
sig3=lf1[lf1==np.min(lf1)]-s3

sig4=lf2[lf2==np.min(lf2)]-s
sig5=lf2[lf2==np.min(lf2)]-s2
sig6=lf2[lf2==np.min(lf2)]-s3

sig7=lf3[lf3==np.min(lf3)]-s
sig8=lf3[lf3==np.min(lf3)]-s2
sig9=lf3[lf3==np.min(lf3)]-s3



fig, axs = plt.subplots(2, 2, figsize=(25,20))
plt.subplots_adjust(wspace = 0.5,hspace=0.5)
axs[0, 0].plot(slp,lf1, linewidth=4,color='b',label="Mean of 100 simulations")
axs[0, 0].plot(slope[35*len(slp):36*len(slp)],l1[35*len(slp):36*len(slp)], linewidth=4,color='r',label="Single simulation")
axs[0, 0].set_title(' 0 to 200 days ')
axs[0, 0].set_ylim([np.min(lf1)-1,np.min(lf1)+10])
axs[0, 0].axhline(y=sig1,linewidth=4,color='black',linestyle='dashed')
axs[0, 0].axhline(y=sig2,linewidth=4,color='black',linestyle='dashed')
axs[0, 0].axhline(y=sig3,linewidth=4,color='black',linestyle='dashed')

axs[0, 1].plot(slp,lf2, linewidth=4,color='b')
axs[0, 1].plot(slope[35*len(slp):36*len(slp)],l1[35*len(slp):36*len(slp)], linewidth=4,color='r')
axs[0, 1].set_title(' 200 to 400 days ')
axs[0, 1].set_ylim([np.min(lf2)-1,np.min(lf2)+10])
axs[0, 1].axhline(y=sig4,linewidth=4,color='black',linestyle='dashed')
axs[0, 1].axhline(y=sig5,linewidth=4,color='black',linestyle='dashed')
axs[0, 1].axhline(y=sig6,linewidth=4,color='black',linestyle='dashed')

axs[1, 0].plot(slp,lf3, linewidth=4,color='b')
axs[1, 0].plot(slope[35*len(slp):36*len(slp)],l3[35*len(slp):36*len(slp)], linewidth=4,color='r')
axs[1, 0].set_title('400 to 600 days ')
axs[1, 0].set_ylim([np.min(lf3)-1,np.min(lf3)+10])
axs[1, 0].axhline(y=sig7,linewidth=4,color='black',linestyle='dashed')
axs[1, 0].axhline(y=sig8,linewidth=4,color='black',linestyle='dashed')
axs[1, 0].axhline(y=sig9,linewidth=4,color='black',linestyle='dashed')

axs[1][0].set_position([0.35,0.1,0.32,0.28])
axs[1, 1].set_visible(False)
for ax in axs.flat:
    ax.set_ylabel('$- \ln \mathcal{L}(F)$')
    ax.set_xlabel(r'$\gamma$')
    ax.set_xlim(np.min(slp),np.max(slp))

# Add legend outside the plot
handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, loc='center left', bbox_to_anchor=(0.5, 0.48))

fig.savefig("sim_cut.pdf")


#__________    DIFFERENT INTERPOLATION    __________________________________


slope=np.loadtxt('pl_cub.dat')[:,0]
ll=np.loadtxt('pl_lin.dat')[:,1]
lc=np.loadtxt('pl_cub.dat')[:,1]
lg=np.loadtxt('pl_gas.dat')[:,1]
lf=np.loadtxt('pl_full_matrix.dat')[:,2]


slp=np.arange(1.5, 2.5, 0.025)

lf_lin = np.zeros(len(slp))
lf_cub = np.zeros(len(slp))
lf_gas = np.zeros(len(slp))
lf_full = np.zeros(len(slp))


for i, sl_ in enumerate(slp):
    
    lf_lin[i] = np.mean(ll[slope == sl_])
    lf_cub[i] = np.mean(lc[slope == sl_])
    lf_gas[i] = np.mean(lg[slope == sl_])
    lf_full[i] = np.mean(lf[slope == sl_])
    

fig, ax = plt.subplots(figsize=(20, 20), nrows=2, ncols=1, gridspec_kw={'height_ratios': [4, 2.9]})
ax[0].plot(slp, lf_lin, linewidth=4,color='orange', label='Linear')
ax[0].plot(slp, lf_cub, linewidth=4,color='b', label='Cubic')
ax[0].plot(slp, lf_gas, linewidth=4,color='r', label='Gaussian')
ax[0].plot(slp, lf_full, linewidth=4,color='g', label='Full Matrix')
ax[0].set_ylabel('$ - \ln \mathcal{L}(F)$', fontsize=30.0)
ax[0].legend(loc='upper center', fontsize=25)
#ax[0].legend(loc='best', fontsize=25)
ax[0].set_xlim(np.min(slp),np.max(slp))
ax[0].grid(True)


gam_lin=np.zeros(len(slope))
gam_cub=np.zeros(len(slope))
gam_gas=np.zeros(len(slope))
gam_full=np.zeros(len(slope))

length=len(slp)


for i in range (0,len(slope),length):
    gam_lin[i]=slope[i:i+length][ll[i:i+length]==np.min(ll[i:i+length])]
    gam_cub[i]=slope[i:i+length][lc[i:i+length]==np.min(lc[i:i+length])]
    gam_gas[i]=slope[i:i+length][lg[i:i+length]==np.min(lg[i:i+length])]
    gam_full[i]=slope[i:i+length][lf[i:i+length]==np.min(lf[i:i+length])]

    
gam_lin=gam_lin[gam_lin!=0]
gam_cub=gam_cub[gam_cub!=0]
gam_gas=gam_gas[gam_gas!=0]
gam_full=gam_full[gam_full!=0]



bins = slp
ax[1].hist(gam_lin, bins, color='orange', histtype='step', lw=6,label='Linear')
ax[1].hist(gam_cub, bins, color='b', histtype='step', lw=6,label='Cubic')
ax[1].hist(gam_gas, bins, color='r', histtype='step', lw=6,label='Gaussian')
ax[1].hist(gam_full, bins, color='g', histtype='step', lw=6,label='Full Matrix')
ax[1].set_xlabel(r'$\gamma$', fontsize=30.0)
ax[1].set_ylabel('Counts', fontsize=35)
ax[1].legend(loc='best', fontsize=25)
ax[1].set_xlim(np.min(slp),np.max(slp))
fig.tight_layout()
plt.savefig('pl_int.pdf')
plt.show()


# For the table: mean best fit gamma with 1 \sigma std. dev

print(round(np.mean(gam_lin),2),round(np.std(gam_lin),2))
print(round(np.mean(gam_cub),2),round(np.std(gam_cub),2))
print(round(np.mean(gam_gas),2),round(np.std(gam_gas),2))
print(round(np.mean(gam_full),2),round(np.std(gam_full),2))


#______________   POWER LAW   _______________________


df = 2
s = -0.5 * chi2.ppf(0.68, df)
s2 = -0.5 * chi2.ppf(0.95, df)
s3 = -0.5 * chi2.ppf(0.997, df)

slope = np.loadtxt('pl_loop.dat')[:,0]
l = np.loadtxt('pl_loop.dat')[:,2]

slp = np.arange(1.5, 2.5, 0.025)

lf = np.zeros(len(slp))
ls = np.zeros(len(slp))

for i, sl_ in enumerate(slp):
    lf[i] = np.mean(l[slope == sl_])
    ls[i] = np.std(l[slope == sl_])

sig = lf[lf == np.min(lf)] - s
sig2 = lf[lf == np.min(lf)] - s2
sig3 = lf[lf == np.min(lf)] - s3

fig, ax = plt.subplots(figsize=(20, 20), nrows=2, ncols=1, gridspec_kw={'height_ratios': [4, 2.9]})
ax[0].plot(slp, lf, linewidth=4, color='blue')
ax[0].axhline(y=sig, linewidth=4, color='black', linestyle='dashed')
ax[0].axhline(y=sig2, linewidth=4, color='black', linestyle='dashed')
ax[0].axhline(y=sig3, linewidth=4, color='black', linestyle='dashed')
ax[0].set_ylim([np.min(lf) - 1, np.min(lf) + 10])
ax[0].set_ylabel('$ - \ln \mathcal{L}(F)$', fontsize=30.0)

gam=np.zeros(len(slope))

length=len(slp)

llf=l.reshape(-1,length)
s=slope.reshape(-1,length)

for i in range (0,len(slope),length):
    gam[i]=slope[i:i+length][l[i:i+length]==np.min(l[i:i+length])]
    
gam=gam[gam!=0]

# calculate percentiles
percentiles = [0.16, 0.84, 0.025, 0.975]
quantiles = np.quantile(gam, percentiles)

bins = slp

# fit normal distribution to data
(mu, sigma) = norm.fit(gam)

# calculate PDF using fitted parameters
pdf = norm.pdf(bins, mu, sigma)


# Rescale the PDF to match the height of the histogram
hist, bins = np.histogram(gam, bins=bins)
bin_width = np.diff(bins)[0]
pdf_max = np.max(pdf)
pdf_scale = np.max(hist) / pdf_max
pdf = pdf * pdf_scale


ax[1].hist(gam, bins, color='r', histtype='bar', ec='black')
ax[1].set_xlabel(r'$\gamma$', fontsize=30.0)
ax[1].set_ylabel('Counts', fontsize=35)
ax[1].plot(bins, pdf, color='black', lw=2)
ax[1].axvline(quantiles[0],linewidth=9,color='black',linestyle='dotted')
ax[1].axvline(quantiles[1],linewidth=9,color='black',linestyle='dotted')
ax[1].axvline(quantiles[2],linewidth=9,color='black',linestyle='dotted')
ax[1].axvline(quantiles[3],linewidth=9,color='black',linestyle='dotted')
fig.tight_layout()
plt.savefig('pl_loop.pdf')
plt.show()

#________________ LOG NOISE LEVEL ___________________________

matplotlib.rc('xtick', labelsize=18) 
matplotlib.rc('ytick', labelsize=25) 
plt.rc('font', size=40)


data = np.loadtxt('pl_log_noise.dat')
slp = data[:, 0]

l = [data[:, i] for i in range(1, 11)]

g1 = np.arange(1.5, 2.5, 0.025) 

lf = np.zeros((10, len(g1)))

for i, g_ in enumerate(g1):
    lf[:, i] = [np.mean(l[j][slp == g_]) for j in range(10)]

length = len(g1)
slp_indices = np.arange(0, len(slp), length)
slp_reshape = slp.reshape(-1, length)
l_reshape = [l[j].reshape(-1, length) for j in range(10)]
min_indices = [np.argmin(l, axis=1) for l in l_reshape]
selected_indices = [np.random.choice(min_idx, size=len(slp_idx), replace=False) for min_idx, slp_idx in zip(min_indices, [slp_indices] * 10)]
gamma = [g_rsh[sel_idx] for g_rsh, sel_idx in zip(slp_reshape, selected_indices)]

slope=2.0 # true  slope

x=np.array([0.5, 0.57009, 0.64565, 0.72654, 0.81283, 0.90485, 1.002, 1.1049, 1.2144, 2])
y=np.mean(gamma,axis=1)/slope # ratio of the fitted slope / true slope 
y_err=(1/slope)*np.std(gamma,axis=1) # error bars on the fitted slope

# Overplotting the results from a single simulation

# Convert the 'gamma' lists into NumPy arrays

gamma = np.array(gamma)

# Calculate the mean of each row
row_means = np.mean(gamma, axis=1)

# Calculate the differences between each element and the mean for each row
diffs = np.abs(gamma - row_means[:, np.newaxis])

# Calculate the combined differences for each index
combined_diffs = np.sum(diffs, axis=0)

# Find the index closest to the mean for each row
closest_indices = np.abs(gamma - row_means[:, np.newaxis]).argmin(axis=1)

# Use the closest_indices to extract the corresponding values from each row

single_sim_slp = (1/slope) * gamma[np.arange(gamma.shape[0]), closest_indices]


# Plot configuration
plt.figure(figsize=(20, 15))

# Plot the data
plt.plot(x, y, marker='o', color='blue', linewidth=4, label='Mean of 100 simulations')
plt.fill_between(x, y-y_err, y+y_err, color='red', alpha=0.4, label=r'$1-\sigma$ deviation of 100 simulations')
plt.plot(x, single_sim_slp, marker='d', color='k', linewidth=4, label='Single Simulation')

# Set the axis labels and scale
plt.xlabel(r'Relative noise level in units of $\sigma_{F}$')
plt.xscale('log')
plt.ylabel(r'$\frac{\gamma_{best-fit} }{ \gamma_{true}}$')

# Set the grid and legend
plt.grid(True)
plt.legend(loc='best', fontsize=22)

# Set the x-axis tick values and labels
plt.xticks([0.5, 0.81283, 1.002,  1.2144, 2], [0.5, 0.81283, 1.002,  1.2144, 2])

# Save and display the plot
plt.savefig("pl_noise.pdf")
plt.show()

# <span style="font-family: 'Verdana', cursive, sans-serif; font-weight: bold; color: green;font-size: 38px;">For The Broken Power law PSD Model</span>

# <span style="font-family: 'Palatino', cursive, sans-serif; font-weight: bold; color: blue;font-size: 30px;">Simple Case</span>

In [ ]:
start = time.time()


n_realizations = 100
results_list = []

for i in range(n_realizations):
    
    np.random.seed(i)

# broken power law parameters

    gamma1=1.
    gamma2=3
    w0=0.3

    alpha1=1.


    N=600 # number of points in simulated light curve
    step=1 # time sampling of simulated light curve

    f_dc=5

#noise term

    noise_level=0.03

    sigma_F=f_dc*noise_level

#Fourier frequencies

    w=2*(np.pi)*np.linspace(1,N,N)/N/step

#Power at each frequency depending on power spectrum model. 


    def power (gamma1,gamma2,w0):
        
        p = np.where(w <= w0, (w / w0)**(-gamma1), (w / w0)**(-gamma2))
                
        return p


#Generate two sets of normally distributed random numbers 
    
    aa = np.random.randn(len(w))
    bb = np.random.randn(len(w))
    
# Light curve in frequency domain  = SQRT(p/2) * (a + b*i)

    f_w = np.sqrt(0.5*power(gamma1,gamma2,w0))*(aa + bb*1j)
    f_w[0]=f_dc * len(w)**0.5

#generating noise

    n=np.ones(len(w))*(sigma_F**2)
    n_w=np.sqrt(0.5*n)*(aa + bb*1.j)
    n_w[0] = np.random.randn(1)*sigma_F

# Generating the total flux (Signal + noise)

    f1_w = alpha1 * f_w
    phi_w = f1_w 
    F_w = phi_w+n_w

    f_t = np.real(np.fft.ifft(f_w[:], norm='ortho'))
    Ft = np.real(np.fft.ifft(F_w[:], norm='ortho'))


    def Sigma_phi(alpha1, gamma1,gamma2,w0):
        Sigma_phi_ = (alpha1**2) * power(gamma1,gamma2,w0)
        return Sigma_phi_


    def cov_mat(alpha1,gamma1,gamma2,w0):
        dt = np.linspace(0, N, N)
        cov=Sigma_phi(alpha1, gamma1,gamma2,w0)

# calculate sigma for all dt values
        sigma = 2 * np.trapz(cov * np.cos(w * dt[:, np.newaxis]), w, axis=1)

# add noise to the first element of sigma
        sigma[0] += sigma_F**2
        
# create the covariance matrix by subtracting the absolute difference between indices
    
        ID = np.arange(sigma.size)
        sigma = sigma[np.abs(ID - ID[:, None])]

        return sigma

    
    def log_like_F (alpha1, gamma1,gamma2,w0):
    
        mu=np.mean(Ft)
      
        data=Ft-mu
    
        inv=np.linalg.inv(cov_mat(alpha1,gamma1,gamma2,w0))
    
        sign,logdet=np.linalg.slogdet(2*np.pi*cov_mat( alpha1, gamma1,gamma2,w0))
    
        lh=-0.5*( logdet + np.transpose(data) @ inv @ data )
    
        return lh



    def mnz(args):
        alpha1, gamma1, gamma2, w0 = args
        return -log_like_F(alpha1, gamma1,gamma2,w0)+log_like_F(1,1,1,1)



    w_max=0.7
    w_min=0.1
    w_step=0.001

    ww=np.arange(w_min,w_max,w_step)  

    diffe=np.zeros(len(ww))
    g=np.zeros(len(ww))
    amp=np.zeros(len(ww))

    results = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.4,1,3,w_), bounds=((0.1,0.9),(1,1),(1.5,5),(w_,w_)))
        for w_ in ww
    )

    for j, res1 in enumerate(results):
        amp[j] = res1.x[0]
        g[j]=float(res1.x[2])
        diffe[j] = -log_like_F(amp[j],1,g[j],ww[j])+log_like_F(1,1,1,1)

    results_list.append({'amp': amp,'g': g, 'diffe': diffe})

    rows=zip(np.round(ww,3),np.round(amp,3),np.round(g,2),np.round(diffe,2))

    with open ('bpl.dat', 'a') as f:
        writer=csv.writer(f, delimiter=' ')
        for i in rows:
            writer.writerow(i)

end = time.time()
print(end - start)

# <span style="font-family: 'Palatino', cursive, sans-serif; font-weight: bold; color: blue;font-size: 30px;">LC analysis in Segments</span>

In [ ]:
start = time.time()


n_realizations = 100
results_list = []

for i in range(n_realizations):
    
    np.random.seed(i)

    gamma1=1.
    gamma2=3
    w0=0.3

    alpha1=1.


    N=600 # number of points in simulated light curve
    step=1 # time sampling of simulated light curve

    f_dc=5

#noise term

    noise_level=0.03

    sigma_F=f_dc*noise_level

#Fourier frequencies

    w=2*(np.pi)*np.linspace(1,N,N)/N/step

#Power at each frequency depending on power spectrum model. 


    def power (gamma1,gamma2,w0):
    
        p = np.where(w <= w0, (w / w0)**(-gamma1), (w / w0)**(-gamma2))
                
        return p


#Generate two sets of normally distributed random numbers 
    
    aa = np.random.randn(len(w))
    bb = np.random.randn(len(w))
    
# Light curve in frequency domain  = SQRT(p/2) * (a + b*i)

    f_w = np.sqrt(0.5*power(gamma1,gamma2,w0))*(aa + bb*1j)
    f_w[0]=f_dc * len(w)**0.5

#generating noise

    n=np.ones(len(w))*(sigma_F**2)
    n_w=np.sqrt(0.5*n)*(aa + bb*1.j)
    n_w[0] = np.random.randn(1)*sigma_F

# Generating the total flux (Signal + noise)

    f1_w = alpha1 * f_w
    phi_w = f1_w 
    F_w = phi_w+n_w

    f_t = np.real(np.fft.ifft(f_w[:], norm='ortho'))
    F_t = np.real(np.fft.ifft(F_w[:], norm='ortho'))

    lim=200
    low=2
    high=3

    Ft=F_t[low*lim:high*lim]

    def Sigma_phi(alpha1, gamma1,gamma2,w0):
        Sigma_phi_ = (alpha1**2) * power(gamma1,gamma2,w0)
        return Sigma_phi_


    def cov_mat(alpha1,gamma1,gamma2,w0):
    
        dt=np.linspace(0,lim,lim)    
        cov=Sigma_phi(alpha1, gamma1,gamma2,w0)
    
        sigma = 2 * np.trapz(cov * np.cos(w * dt[:, np.newaxis]), w, axis=1)
           
# adding noise to the covariance matrix     
    
        sigma[0]+=sigma_F**2   
        ID = np.arange(sigma.size)
        sigma= sigma[np.abs(ID - ID[:,None])]
    
        return sigma
    
    def log_like_F (alpha1, gamma1,gamma2,w0):
    
        mu=np.mean(Ft)
      
        data=Ft-mu
    
        inv=np.linalg.inv(cov_mat(alpha1,gamma1,gamma2,w0))
    
        sign,logdet=np.linalg.slogdet(2*np.pi*cov_mat( alpha1, gamma1,gamma2,w0))
    
        lh=-0.5*( logdet + np.transpose(data) @ inv @ data )
    
        return lh


    def mnz(args):
            alpha1, gamma1, gamma2, w0 = args
            return -log_like_F(alpha1, gamma1,gamma2,w0)+log_like_F(1,1,1,1)



    w_max=0.7
    w_min=0.1
    w_step=0.001

    ww=np.arange(w_min,w_max,w_step)  

    diffe=np.zeros(len(ww))
    g=np.zeros(len(ww))
    amp=np.zeros(len(ww))

    results = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.4,1,3,w_), bounds=((0.05,0.9),(1,1),(1.5,5),(w_,w_)))
        for w_ in ww
    )

    for j, res1 in enumerate(results):
        amp[j] = res1.x[0]
        g[j]=float(res1.x[2])
        diffe[j] = -log_like_F(amp[j],1,g[j],ww[j])+log_like_F(1,1,1,1)

    results_list.append({'amp': amp,'g': g, 'diffe': diffe})

    rows=zip(np.round(ww,3),np.round(amp,3),np.round(g,2),np.round(diffe,2))

 #   with open ('b1.dat', 'a') as f:
 #   with open ('b2.dat', 'a') as f:
    with open ('b3.dat', 'a') as f:
        writer=csv.writer(f, delimiter=' ')
        for i in rows:
            writer.writerow(i)

end = time.time()
print(end - start)

# <span style="font-family: 'Palatino', cursive, sans-serif; font-weight: bold; color: blue;font-size: 30px;">Effect of Gaps and Interpolation Schemes</span>

In [ ]:
start = time.time()


n_realizations = 100
results_list = []

for i in range(n_realizations):
    
    np.random.seed(i)

    gamma1=1.
    gamma2=3
    w0=0.3

    alpha1=1.

    
    N=600 # number of points in simulated light curve
    step=1 # time sampling of simulated light curve
    
    f_dc=0.1
    
    #noise term
    
    noise_level=0.03
    
    sigma_F=f_dc*noise_level
    
    #Fourier frequencies
    
    w=2*(np.pi)*np.linspace(1,N,N)/N/step
    
    #Power at each frequency depending on power spectrum model. 
    
    
    def power (gamma1,gamma2,w0):
    
        p = np.where(w <= w0, (w / w0)**(-gamma1), (w / w0)**(-gamma2))
                
        return p
    
    
    
    #Generate two sets of normally distributed random numbers 
        
    aa = np.random.randn(len(w))
    bb = np.random.randn(len(w))
        
    # Light curve in frequency domain  = SQRT(p/2) * (a + b*i)
    
    f_w = np.sqrt(0.5*power(gamma1,gamma2,w0))*(aa + bb*1j)
    
    f_w[0]=f_dc * len(w)**0.5
    
    #generating noise
    
    n=np.ones(len(w))*(sigma_F**2)
    
    n_w=np.sqrt(0.5*n)*(aa + bb*1.j)
    
    n_w[0] = np.random.randn(1)*sigma_F
    
    # Generating the total flux (Signal + noise)
    
    f1_w = alpha1 * f_w
    phi_w = f1_w 
    F_w = phi_w+n_w
    
    f_t = np.real(np.fft.ifft(f_w[:], norm='ortho'))
    
    F_t = np.real(np.fft.ifft(F_w[:], norm='ortho'))

# section to remove points and interpolation

    T=np.linspace(0,N,N)

    # Calculate the number of points to remove
    num_points = int(len(T) * 0.1)

    # Randomly select indices to remove with gaps ranging from 1 to 10 points
    remove_indices = []
    num_removed = 0
    while num_removed < num_points:
        index = np.random.randint(0, len(T) - 10)
        gap = np.random.randint(1, 11)
        remove_indices.extend(range(index, min(index + gap, len(T))))
        num_removed += gap

    # Remove the selected data points from the time series and the flux array
    t4 = np.delete(T, remove_indices)
    Ft4 = np.delete(F_t, remove_indices)

#    interpolation schemes

   # f = interpolate.interp1d(t4,Ft4,kind='linear',fill_value='extrapolate')
  #  f = Rbf(t4,Ft4,function='cubic',fill_value=1e+10)
 

 # # This section is for the Gaussian Process Interpolation 
  
 # Define the Kriging model with Matern kernel
    
    kernel = Matern(length_scale=0.2, nu=1.5)
    model = GaussianProcessRegressor(kernel=kernel)
     
     # Train the model
     
    model.fit(t4.reshape(-1, 1), Ft4)
    Ft=model.predict(T.reshape(-1, 1))


    def Sigma_phi(alpha1, gamma1,gamma2,w0):
            Sigma_phi_ = (alpha1**2) * power(gamma1,gamma2,w0)
            return Sigma_phi_


    def cov_mat(alpha1,gamma1,gamma2,w0):
        dt = np.linspace(0, N, N)
        cov=Sigma_phi(alpha1, gamma1,gamma2,w0)

# calculate sigma for all dt values
        sigma = 2 * np.trapz(cov * np.cos(w * dt[:, np.newaxis]), w, axis=1)

# add noise to the first element of sigma
        sigma[0] += sigma_F**2
        
# create the covariance matrix by subtracting the absolute difference between indices
    
        ID = np.arange(sigma.size)
        sigma = sigma[np.abs(ID - ID[:, None])]

        return sigma

    
    def log_like_F (alpha1, gamma1,gamma2,w0):
    
        mu=np.mean(Ft)
      
        data=Ft-mu
    
        inv=np.linalg.inv(cov_mat(alpha1,gamma1,gamma2,w0))
    
        sign,logdet=np.linalg.slogdet(2*np.pi*cov_mat( alpha1, gamma1,gamma2,w0))
    
        lh=-0.5*( logdet + np.transpose(data) @ inv @ data )
    
        return lh



    def mnz(args):
        alpha1, gamma1, gamma2, w0 = args
        return -log_like_F(alpha1, gamma1,gamma2,w0)+log_like_F(1,1,1,1)



    w_max=0.7
    w_min=0.1
    w_step=0.001

    ww=np.arange(w_min,w_max,w_step)  

    diffe=np.zeros(len(ww))
    g=np.zeros(len(ww))
    amp=np.zeros(len(ww))

    results = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.4,1,3,w_), bounds=((0.005,5),(1,1),(1.5,10),(w_,w_)))
        for w_ in ww
    )

    for j, res1 in enumerate(results):
        amp[j] = res1.x[0]
        g[j]=float(res1.x[2])
        diffe[j] = -log_like_F(amp[j],1,g[j],ww[j])+log_like_F(1,1,1,1)

    results_list.append({'amp': amp,'g': g, 'diffe': diffe})

    rows=zip(np.round(ww,3),np.round(amp,3),np.round(g,2),np.round(diffe,2))

 #   with open ('bpl_lin.dat', 'a') as f:
 #   with open ('bpl_cub.dat', 'a') as f:
    with open ('bpl_gas.dat', 'a') as f:
        writer=csv.writer(f, delimiter=' ')
        for i in rows:
            writer.writerow(i)

end = time.time()
print(end - start)

# <span style="font-family: 'Palatino', cursive, sans-serif; font-weight: bold; color: blue;font-size: 30px;">Effect of Incorrect Noise Modeling</span>

In [ ]:
start = time.time()


n_realizations = 100


for i in range(n_realizations):
    
    np.random.seed(i)

# broken power law parameters

    gamma1=1.
    gamma2=3
    w0=0.3

    alpha1=1.


    N=600 # number of points in simulated light curve
    step=1 # time sampling of simulated light curve

    f_dc=0.1

#noise term

    noise_level=0.03
    sigma_F=f_dc*noise_level

#Fourier frequencies

    w=2*(np.pi)*np.linspace(1,N,N)/N/step

#Power at each frequency depending on power spectrum model. 


    def power (gamma1,gamma2,w0):
        
        p = np.where(w <= w0, (w / w0)**(-gamma1), (w / w0)**(-gamma2))
                
        return p


#Generate two sets of normally distributed random numbers 
    
    aa = np.random.randn(len(w))
    bb = np.random.randn(len(w))
    
# Light curve in frequency domain  = SQRT(p/2) * (a + b*i)

    f_w = np.sqrt(0.5*power(gamma1,gamma2,w0))*(aa + bb*1j)
    f_w[0]=f_dc * len(w)**0.5

#generating noise

    n=np.ones(len(w))*(sigma_F**2)
    n_w=np.sqrt(0.5*n)*(aa + bb*1.j)
    n_w[0] = np.random.randn(1)*sigma_F

# Generating the total flux (Signal + noise)

    f1_w = alpha1 * f_w
    phi_w = f1_w 
    F_w = phi_w+n_w

    f_t = np.real(np.fft.ifft(f_w[:], norm='ortho'))
    F_t = np.real(np.fft.ifft(F_w[:], norm='ortho'))


    def Sigma_phi(alpha1, gamma1,gamma2,w0):
        Sigma_phi_ = (alpha1**2) * power(gamma1,gamma2,w0)
        return Sigma_phi_
    
    
    def cov_mat(alpha1,gamma1,gamma2,w0,sigma_F):
        
        dt = np.linspace(0, N, N)
        cov=Sigma_phi(alpha1, gamma1,gamma2,w0)
        
        # calculate sigma for all dt values
        sigma = 2 * np.trapz(cov * np.cos(w * dt[:, np.newaxis]), w, axis=1)
    
        # add noise to the first element of sigma
        sigma[0] += sigma_F**2
                
        # create the covariance matrix by subtracting the absolute difference between indices
            
        ID = np.arange(sigma.size)
        sigma = sigma[np.abs(ID - ID[:, None])]
    
        return sigma
    
    
        
    def log_like_F (alpha1, gamma1,gamma2,w0,sigma_F):
        
        mu=np.mean(F_t)
          
        data=F_t-mu
        
        inv=np.linalg.inv(cov_mat(alpha1,gamma1,gamma2,w0,sigma_F))
        
        sign,logdet=np.linalg.slogdet(2*np.pi*cov_mat( alpha1, gamma1,gamma2,w0,sigma_F))
        
        lh=-0.5*( logdet + np.transpose(data) @ inv @ data )
        
        return lh
    
    
    
    def mnz(args):
            alpha1, gamma1, gamma2, w0, sigma_F = args
            return -log_like_F(alpha1, gamma1,gamma2,w0,sigma_F)+log_like_F(1,1,1,1,sigma_F)
    
    
    
    w_max=0.7
    w_min=0.1
    w_step=0.001
    
    ww=np.arange(w_min,w_max,w_step)  
    
    # Noise level from 0.5 to 2.0 in 5 log steps
    
    ss1, ss2, ss3, ss4, ss5, ss6, ss7, ss8, ss9, ss10 = np.array([0.5, 0.57009, 0.64565, 0.72654, 0.81283, 0.90485, 1.002, 1.1049, 1.2144, 2]) * sigma_F
    diffe1, diffe2, diffe3, diffe4, diffe5, diffe6, diffe7, diffe8, diffe9, diffe10  = np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww))
    gg1, gg2, gg3, gg4, gg5, gg6, gg7, gg8, gg9, gg10 = np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww))
    amp1, amp2, amp3, amp4, amp5, amp6, amp7, amp8, amp9, amp10 = np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww)), np.zeros(len(ww))
    
    
    
    
    results = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.4,1,3,w_,ss1), bounds=((0.05,5),(1,1),(1.,10),(w_,w_),(ss1,ss1)))
        for w_ in ww
    )
    
    for j, res1 in enumerate(results):
        amp1[j] = res1.x[0]
        gg1[j]=float(res1.x[2])
        diffe1[j] = -log_like_F(amp1[j],1,gg1[j],ww[j],ss1)+log_like_F(1,1,1,1,ss1)
        
    results2 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.4,1,3,w_,ss2), bounds=((0.05,5),(1,1),(1.,10),(w_,w_),(ss2,ss2)))
        for w_ in ww
    )
    
    for j, res2 in enumerate(results2):
        amp2[j] = res2.x[0]
        gg2[j]=float(res2.x[2])
        diffe2[j] = -log_like_F(amp2[j],1,gg2[j],ww[j],ss2)+log_like_F(1,1,1,1,ss2)
        
    results3 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.4,1,3,w_,ss3), bounds=((0.05,5),(1,1),(1.,10),(w_,w_),(ss3,ss3)))
        for w_ in ww
    )
    
    for j, res3 in enumerate(results3):
        amp3[j] = res3.x[0]
        gg3[j]=float(res3.x[2])
        diffe3[j] = -log_like_F(amp3[j],1,gg3[j],ww[j],ss3)+log_like_F(1,1,1,1,ss3)
        
        
    results4 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.4,1,3,w_,ss4), bounds=((0.005,150),(1,1),(1.,10),(w_,w_),(ss4,ss4)))
        for w_ in ww
    )
    
    for j, res4 in enumerate(results4):
        amp4[j] = res4.x[0]
        gg4[j]=float(res4.x[2])
        diffe4[j] = -log_like_F(amp4[j],1,gg4[j],ww[j],ss4)+log_like_F(1,1,1,1,ss4)
        
        
    results5 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.4,1,3,w_,ss5), bounds=((0.005,150),(1,1),(1.,10),(w_,w_),(ss5,ss5)))
        for w_ in ww
    )
    
    for j, res5 in enumerate(results5):
        amp5[j] = res5.x[0]
        gg5[j]=float(res5.x[2])
        diffe5[j] = -log_like_F(amp5[j],1,gg5[j],ww[j],ss5)+log_like_F(1,1,1,1,ss5)
        
        
    results6 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.4,1,3,w_,ss6), bounds=((0.05,5),(1,1),(1.,10),(w_,w_),(ss6,ss6)))
        for w_ in ww
    )
    
    for j, res6 in enumerate(results6):
        amp6[j] = res6.x[0]
        gg6[j]=float(res6.x[2])
        diffe6[j] = -log_like_F(amp6[j],1,gg6[j],ww[j],ss6)+log_like_F(1,1,1,1,ss6)
        
    results7 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.4,1,3,w_,ss7), bounds=((0.05,5),(1,1),(1.,10),(w_,w_),(ss7,ss7)))
        for w_ in ww
    )
    
    for j, res7 in enumerate(results7):
        amp7[j] = res7.x[0]
        gg7[j]=float(res7.x[2])
        diffe7[j] = -log_like_F(amp7[j],1,gg7[j],ww[j],ss7)+log_like_F(1,1,1,1,ss7)
        
    results8 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.4,1,3,w_,ss8), bounds=((0.05,5),(1,1),(1.,10),(w_,w_),(ss8,ss8)))
        for w_ in ww
    )
    
    for j, res8 in enumerate(results8):
        amp8[j] = res8.x[0]
        gg8[j]=float(res8.x[2])
        diffe8[j] = -log_like_F(amp8[j],1,gg8[j],ww[j],ss8)+log_like_F(1,1,1,1,ss8)
        
        
    results9 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.4,1,3,w_,ss9), bounds=((0.005,150),(1,1),(1.,10),(w_,w_),(ss9,ss9)))
        for w_ in ww
    )
    
    for j, res9 in enumerate(results9):
        amp9[j] = res9.x[0]
        gg9[j]=float(res9.x[2])
        diffe9[j] = -log_like_F(amp9[j],1,gg9[j],ww[j],ss9)+log_like_F(1,1,1,1,ss9)
        
        
    results10 = Parallel(n_jobs=multiprocessing.cpu_count())(
        delayed(minimize)(mnz, (0.4,1,3,w_,ss10), bounds=((0.005,150),(1,1),(1.,10),(w_,w_),(ss10,ss10)))
        for w_ in ww
    )
    
    for j, res10 in enumerate(results10):
        amp10[j] = res10.x[0]
        gg10[j]=float(res10.x[2])
        diffe10[j] = -log_like_F(amp10[j],1,gg10[j],ww[j],ss10)+log_like_F(1,1,1,1,ss10)
    
    
    rows=zip(np.round(ww,3),np.round(gg1,2),np.round(diffe1,2),np.round(gg2,2),
              np.round(diffe2,2),np.round(gg3,2),np.round(diffe3,2),np.round(gg4,2),
                      np.round(diffe4,2),np.round(gg5,2),np.round(diffe5,2),
                      np.round(gg6,2),np.round(diffe6,2),np.round(gg7,2),
                                np.round(diffe7,2),np.round(gg8,2),np.round(diffe8,2),np.round(gg9,2),
                                        np.round(diffe9,2),np.round(gg10,2),np.round(diffe10,2))
    
    
    
    with open ('bpl_dif_noise.dat', 'a') as f:
        writer=csv.writer(f, delimiter=' ')
        for i in rows:
               writer.writerow(i)

end = time.time()
print(end - start)


# <span style="font-family: 'Century Gothic', cursive, sans-serif; font-weight: bold; color: purple;font-size: 30px;">Template for Plots</span>

In [ ]:

#______________ BROKEN POWER LAW LOOP __________________________

matplotlib.rc('xtick', labelsize=35) 
matplotlib.rc('ytick', labelsize=35) 
plt.rc('font', size=45)


df = 3
s = -0.5 * chi2.ppf(0.68, df)
s2 = -0.5 * chi2.ppf(0.95, df)
s3 = -0.5 * chi2.ppf(0.997, df)

wbk=np.loadtxt('bpl.dat')[:,0]

g=np.loadtxt('bpl.dat')[:,2]
l=np.loadtxt('bpl.dat')[:,3]

w_max=0.7
w_min=0.1
w_step=0.001
ww=np.round(np.arange(w_min,w_max,w_step),3) 

lf = np.zeros(len(ww))
gg = np.zeros(len(ww))



for i, ww_ in enumerate(ww):
    lf[i] = np.mean(l[wbk == ww_])
    gg[i] = np.mean(g[wbk == ww_])
    

sig1=lf[lf==np.min(lf)]-s
sig2=lf[lf==np.min(lf)]-s2
sig3=lf[lf==np.min(lf)]-s3


# For the \omega bk Plot:-


fig, ax = plt.subplots(figsize=(20, 20))
ax.plot(ww, lf, linewidth=4, color='blue')
ax.axhline(y=sig1, linewidth=4, color='black', linestyle='dashed')
ax.axhline(y=sig2, linewidth=4, color='black', linestyle='dashed')
ax.axhline(y=sig3, linewidth=4, color='black', linestyle='dashed')
ax.set_ylim([np.min(lf) - 1, np.min(lf) + 10])
ax.set_ylabel('$ - \ln \mathcal{L}(F)$')
ax.set_xlabel('$\omega_{bk}$ (rad/day)')
plt.savefig('bpl_loop_wbk.pdf')
plt.show()

# For the \gamma2 Plot:-


fig, ax = plt.subplots(figsize=(20, 20))
ax.plot(gg, lf, linewidth=4, color='blue')
ax.axhline(y=sig1, linewidth=4, color='black', linestyle='dashed')
ax.axhline(y=sig2, linewidth=4, color='black', linestyle='dashed')
ax.axhline(y=sig3, linewidth=4, color='black', linestyle='dashed')
ax.set_ylim([np.min(lf) - 1, np.min(lf) + 10])
ax.set_ylabel('$ - \ln \mathcal{L}(F)$')
ax.set_xlabel('$\gamma_{2}$')
plt.savefig('bpl_loop_slope.pdf')
plt.show()


matplotlib.rc('xtick', labelsize=45) 
matplotlib.rc('ytick', labelsize=45) 
plt.rc('font', size=45)

# For the Contour Plot:  
    
length=len(ww)
    
brk_indices = np.arange(0, len(wbk), length)
wbk_reshape = wbk.reshape(-1, length)
l_reshape = l.reshape(-1, length)
min_indices = np.argmin(l_reshape, axis=1)
selected_indices = np.random.choice(min_indices, size=len(brk_indices))
brk = wbk_reshape[np.arange(len(brk_indices)), selected_indices]  

gf_indices = np.arange(0, len(wbk), length)
gf=g.reshape(-1,length)
g2=gf[np.arange(len(brk_indices)), selected_indices] 


# Define confidence levels
df = 2
s = 0.5 * chi2.ppf(0.68, df)
s2 = 0.5 * chi2.ppf(0.95, df)
s3 = 0.5 * chi2.ppf(0.997, df)

Level=[s,s2,s3]

# Calculate mean and covariance matrix
data = np.column_stack((brk, g2))
mean=(np.mean(data,axis=0))
cov = np.cov(data, rowvar=False)


# # Create figure and axis
fig, ax = plt.subplots(figsize=(20, 20))

# # Create colorbar
cmap = plt.get_cmap('Set1')
vmin = lf.min()
vmax = lf.max()
norm = mcolors.TwoSlopeNorm(vmax=vmax, vcenter=(vmax+vmin)//2, vmin=vmin)
sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])

for i, conf_level in enumerate(Level):
    lambda_, v = np.linalg.eig(cov)
    lambda_ = np.sqrt(lambda_)
    ell1 = Ellipse(xy=mean, width=lambda_[0]*np.sqrt(Level[0])*2,
                  height=lambda_[1]*np.sqrt(Level[0])*2,
                  angle=np.rad2deg(np.arccos(v[0, 0])), fill=True, alpha=1., facecolor=cmap(0))
    ell2 = Ellipse(xy=mean, width=lambda_[0]*np.sqrt(Level[1])*2,
                  height=lambda_[1]*np.sqrt(Level[1])*2,
                  angle=np.rad2deg(np.arccos(v[0, 0])), fill=True, alpha=0.2, facecolor=cmap(1))
    ell3 = Ellipse(xy=mean, width=lambda_[0]*np.sqrt(Level[2])*2,
                  height=lambda_[1]*np.sqrt(Level[2])*2,
                  angle=np.rad2deg(np.arccos(v[0, 0])), fill=True, alpha=0.2, facecolor=cmap(9))
    
    

    ax.add_artist(ell1)
    ax.add_artist(ell2)
    ax.add_artist(ell3)


# Set plot limits
ax.set_xlim(0.1,0.56)
ax.set_ylim(1.5,4.2)

# Set background color
ax.set_facecolor('white')

# Add colorbar
cbar = plt.colorbar(sm)

# Set xticks and yticks
ax.set_xlabel('$\omega_{bk}$ (rad/day)')
ax.set_ylabel('$\gamma_{2}$')
cbar.set_label('$- \ln \mathcal{L}(F)$')

# Annotate a point with a marker and label
point_x = 0.3
point_y = 3.
ax.plot(point_x, point_y, marker='*', markersize=35, color='k')

# Show plot
plt.show()
fig.savefig('bpl_contour.pdf')


#_______________ BROKEN POWER LAW SEGMENTS_____________________

matplotlib.rc('xtick', labelsize=25) 
matplotlib.rc('ytick', labelsize=25) 
plt.rc('font', size=25)



wbk=np.loadtxt('b1.dat')[:,0]
g1, g2, g3 = [np.loadtxt(f)[:, 2] for f in ['b1.dat', 'b2.dat', 'b3.dat']]
l1, l2, l3 = [np.loadtxt(f)[:, 3] for f in ['b1.dat', 'b2.dat', 'b3.dat']]

w_max, w_min, w_step = 0.7, 0.1, 0.001
ww = np.round(np.arange(w_min, w_max, w_step), 3) 

lf1, lf2, lf3, gg1, gg2, gg3 = [np.zeros(len(ww)) for i in range(6)]

for i, ww_ in enumerate(ww):
    lf1[i], lf2[i], lf3[i] = [np.mean(l[wbk == ww_]) for l in [l1, l2, l3]]
    gg1[i], gg2[i], gg3[i] = [np.mean(g[wbk == ww_]) for g in [g1, g2, g3]]


length = len(ww)
brk_indices = np.arange(0, len(wbk), length)
wbk_reshape = wbk.reshape(-1, length)
l_reshape = [l.reshape(-1, length) for l in [l1, l2, l3]]
min_indices = [np.argmin(l, axis=1) for l in l_reshape]
selected_indices = [np.random.choice(min_idx, size=len(brk_idx)) for min_idx, brk_idx in zip(min_indices, [brk_indices]*3)]
brk = [wbk_rsh[np.arange(len(brk_idx)), sel_idx] for wbk_rsh, sel_idx, brk_idx in zip([wbk_reshape]*3, selected_indices, [brk_indices]*3)]
brk1, brk2, brk3 = brk



g_indices = np.arange(0, len(wbk), length)
g_reshape = [g.reshape(-1,length) for g in [g1,g2,g3]]
gamma=[g_rsh[np.arange(len(brk_idx)), sel_idx] for g_rsh, sel_idx, brk_idx in zip(g_reshape, selected_indices, [g_indices]*3)]



gamma1, gamma2, gamma3 = gamma


# Define confidence levels
df = 3
s = 0.5 * chi2.ppf(0.68, df)
s2 = 0.5 * chi2.ppf(0.95, df)
s3 = 0.5 * chi2.ppf(0.997, df)

Level=[s,s2,s3]

# Calculate mean and covariance matrices
data_list = [np.column_stack((brk1, gamma1)), np.column_stack((brk2, gamma2)), 
             np.column_stack((brk3, gamma3))]
mean_list = [np.mean(data, axis=0) for data in data_list]
cov_list = [np.cov(data, rowvar=False) for data in data_list]

point_x = 0.3
point_y = 3.

# Create colorbar
cmap = plt.get_cmap('Set1')
fig, axs = plt.subplots(2,2, figsize=(20, 20))
for i, (lf, cov, mean, gg) in enumerate(zip([lf1, lf2, lf3], [cov_list[0], cov_list[1], cov_list[2]], mean_list, [gg1, gg2, gg3])):
    # Create figure and axis


    # Create colorbar
    vmin = lf.min()
    vmax = lf.max()
    norm = mcolors.TwoSlopeNorm(vmax=vmax, vcenter=(vmax+vmin)//2, vmin=vmin)
    sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])

    for j, conf_level in enumerate(Level):
        lambdai_, v = np.linalg.eig(cov)
        lambdai_ = np.sqrt(lambdai_)
        ell1 = Ellipse(xy=mean, width=lambdai_[0]*np.sqrt(Level[0])*2,
                        height=lambdai_[1]*np.sqrt(Level[0])*2,
                        angle=np.rad2deg(np.arccos(v[0, 0])), fill=True, alpha=1., facecolor=cmap(0))
        ell2 = Ellipse(xy=mean, width=lambdai_[0]*np.sqrt(Level[1])*2,
                        height=lambdai_[1]*np.sqrt(Level[1])*2,
                        angle=np.rad2deg(np.arccos(v[0, 0])), fill=True, alpha=0.2, facecolor=cmap(1))
        ell3 = Ellipse(xy=mean, width=lambdai_[0]*np.sqrt(Level[2])*2,
                        height=lambdai_[1]*np.sqrt(Level[2])*2,
                        angle=np.rad2deg(np.arccos(v[0, 0])), fill=True, alpha=0.2, facecolor=cmap(9))
        # Add Ellipse artists to subplot
        if i == 0:
                
            axs[0, 0].add_artist(ell1)
            axs[0, 0].add_artist(ell2)
            axs[0, 0].add_artist(ell3)
            axs[0, 0].set_xlim(0.1, 0.7)
            axs[0, 0].set_ylim(1.5, 4.5)
            axs[0, 0].plot(point_x, point_y, marker='*', markersize=25, color='k')
            axs[0, 0].set_title('0 to 200 days ')
            axs[0, 0].set_xlabel('$\omega_{bk}$ (rad/day)')
            axs[0, 0].set_ylabel('$\gamma_{2}$')

          
        elif i == 1:
            axs[0, 1].add_artist(ell1)
            axs[0, 1].add_artist(ell2)
            axs[0, 1].add_artist(ell3)
            axs[0, 1].set_xlim(0.1, 0.7)
            axs[0, 1].set_ylim(1.5, 4.5)
            axs[0, 1].set_xlabel('$\omega_{bk}$ (rad/day)')
            axs[0, 1].set_ylabel('$\gamma_{2}$')

            axs[0, 1].set_title('200 to 400 days ')
            axs[0, 1].plot(point_x, point_y, marker='*', markersize=25, color='k')

        
        elif i == 2:
            axs[1, 0].add_artist(ell1)
            axs[1, 0].add_artist(ell2)
            axs[1, 0].add_artist(ell3)
            axs[1, 0].set_xlim(0.1, 0.7)
            axs[1, 0].set_ylim(1.5, 4.5)
            axs[1, 0].set_title('400 to 600 days ')
            axs[1, 0].set_xlabel('$\omega_{bk}$ (rad/day)')
            axs[1, 0].set_ylabel('$\gamma_{2}$')
            axs[1, 0].plot(point_x, point_y, marker='*', markersize=25, color='k')
            axs[1, 0].set_position([0.35,0.1,0.36,0.34])
            axs[1, 1].set_visible(False)
            
            
  
        
cbar = fig.colorbar(sm)
cbar.ax.set_position([0.75,0.1,0.52,0.28])
cbar.set_label('$- \ln \mathcal{L}(F)$')
            
# Show plot
plt.show()
fig.savefig('bpl_seg.pdf')  

#_____________ BROKEN POWER LAW INTERPOLATION ______________________


matplotlib.rc('xtick', labelsize=25) 
matplotlib.rc('ytick', labelsize=25) 
plt.rc('font', size=25)


wbk=np.loadtxt('bpl_full_matrix.dat')[:,0]
g1, g2, g3, g4 = [np.loadtxt(f)[:, 2] for f in ['bpl_lin.dat', 'bpl_cub.dat', 
                                            'bpl_gas.dat','bpl_full_matrix.dat']]
l1, l2, l3, l4 = [np.loadtxt(f)[:, 3] for f in ['bpl_lin.dat', 'bpl_cub.dat', 
                                            'bpl_gas.dat','bpl_full_matrix.dat']]

w_max, w_min, w_step = 0.7, 0.1, 0.001
ww = np.round(np.arange(w_min, w_max, w_step), 3) 

lf1, lf2, lf3,lf4, gg1, gg2, gg3, gg4 = [np.zeros(len(ww)) for i in range(8)]

for i, ww_ in enumerate(ww):
    lf1[i], lf2[i], lf3[i], lf4[i] = [np.mean(l[wbk == ww_]) for l in [l1, l2, l3, l4]]
    gg1[i], gg2[i], gg3[i], gg4[i] = [np.mean(g[wbk == ww_]) for g in [g1, g2, g3, g4]]


length = len(ww)
brk_indices = np.arange(0, len(wbk), length)
wbk_reshape = wbk.reshape(-1, length)
l_reshape = [l.reshape(-1, length) for l in [l1, l2, l3, l4]]
min_indices = [np.argmin(l, axis=1) for l in l_reshape]
selected_indices = [np.random.choice(min_idx, size=len(brk_idx)) for min_idx, brk_idx in zip(min_indices, [brk_indices]*4)]
brk = [wbk_rsh[np.arange(len(brk_idx)), sel_idx] for wbk_rsh, sel_idx, brk_idx in zip([wbk_reshape]*4, selected_indices, [brk_indices]*4)]
brk1, brk2, brk3, brk4 = brk



g_indices = np.arange(0, len(wbk), length)
g_reshape = [g.reshape(-1,length) for g in [g1,g2,g3, g4]]
gamma=[g_rsh[np.arange(len(brk_idx)), sel_idx] for g_rsh, sel_idx, brk_idx in zip(g_reshape, selected_indices, [g_indices]*4)]



gamma1, gamma2, gamma3, gamma4 = gamma


# Define confidence levels
df = 3
s = 0.5 * chi2.ppf(0.68, df)
s2 = 0.5 * chi2.ppf(0.95, df)
s3 = 0.5 * chi2.ppf(0.997, df)

Level=[s,s2,s3]

# Calculate mean and covariance matrices
data_list = [np.column_stack((brk1, gamma1)), np.column_stack((brk2, gamma2)), 
              np.column_stack((brk3, gamma3)), np.column_stack((brk4, gamma4))]
mean_list = [np.mean(data, axis=0) for data in data_list]
cov_list = [np.cov(data, rowvar=False) for data in data_list]

point_x = 0.3
point_y = 3.

# Create colorbar
cmap = plt.get_cmap('Set1')
fig, axs = plt.subplots(3,2, figsize=(20, 25))
plt.subplots_adjust(hspace=0.5)
plt.subplots_adjust(wspace=0.3)
for i, (lf, cov, mean, gg) in enumerate(zip([lf1, lf2, lf3, lf4], [cov_list[0], cov_list[1], cov_list[2], cov_list[3]], mean_list, [gg1, gg2, gg3, gg4])):
    # Create figure and axis


    # Create colorbar
    vmin = lf.min()
    vmax = lf.max()
    norm = mcolors.TwoSlopeNorm(vmax=vmax, vcenter=(vmax+vmin)//2, vmin=vmin)
    sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])

    for j, conf_level in enumerate(Level):
        lambdai_, v = np.linalg.eig(cov)
        lambdai_ = np.sqrt(lambdai_)
        ell1 = Ellipse(xy=mean, width=lambdai_[0]*np.sqrt(Level[0])*2,
                        height=lambdai_[1]*np.sqrt(Level[0])*2,
                        angle=np.rad2deg(np.arccos(v[0, 0])), fill=True, alpha=1., facecolor=cmap(0))
        ell2 = Ellipse(xy=mean, width=lambdai_[0]*np.sqrt(Level[1])*2,
                        height=lambdai_[1]*np.sqrt(Level[1])*2,
                        angle=np.rad2deg(np.arccos(v[0, 0])), fill=True, alpha=0.2, facecolor=cmap(1))
        ell3 = Ellipse(xy=mean, width=lambdai_[0]*np.sqrt(Level[2])*2,
                        height=lambdai_[1]*np.sqrt(Level[2])*2,
                         angle=np.rad2deg(np.arccos(v[0, 0])), fill=True, alpha=0.2, facecolor=cmap(9))
#        Add Ellipse artists to subplot
        if i == 0:
                
            axs[0, 0].add_artist(ell1)
            axs[0, 0].add_artist(ell2)
            axs[0, 0].add_artist(ell3)
            axs[0, 0].set_xlim(0.1, 0.7)
            axs[0, 0].set_ylim(1.5, 6.5)
            axs[0, 0].plot(point_x, point_y, marker='*', markersize=25, color='k')
            axs[0, 0].set_title('Linear')
            axs[0, 0].set_xlabel('$\omega_{bk}$ (rad/day)')
            axs[0, 0].set_ylabel('$\gamma_{2}$')

          
        elif i == 1:
            axs[0, 1].add_artist(ell1)
            axs[0, 1].add_artist(ell2)
            axs[0, 1].add_artist(ell3)
            axs[0, 1].set_xlim(0.1, 0.7)
            axs[0, 1].set_ylim(1.5, 6.5)
            axs[0, 1].set_xlabel('$\omega_{bk}$ (rad/day)')
            axs[0, 1].set_ylabel('$\gamma_{2}$')
            axs[0, 1].set_title('Cubic')
            axs[0, 1].plot(point_x, point_y, marker='*', markersize=25, color='k')

        
        elif i == 2:
            axs[1, 0].add_artist(ell1)
            axs[1, 0].add_artist(ell2)
            axs[1, 0].add_artist(ell3)
            axs[1, 0].set_xlim(0.1, 0.6)
            axs[1, 0].set_ylim(2., 4.5)
            axs[1, 0].set_title('Gaussian')
            axs[1, 0].set_xlabel('$\omega_{bk}$ (rad/day)')
            axs[1, 0].set_ylabel('$\gamma_{2}$')
            axs[1, 0].plot(point_x, point_y, marker='*', markersize=25, color='k')
            
            
        elif i == 3:
            axs[1, 1].add_artist(ell1)
            axs[1, 1].add_artist(ell2)
            axs[1, 1].add_artist(ell3)
            axs[1, 1].set_xlim(0.1, 0.6)
            axs[1, 1].set_ylim(1.5, 4.)
            axs[1, 1].set_title('Full Matrix')
            axs[1, 1].set_xlabel('$\omega_{bk}$ (rad/day)')
            axs[1, 1].set_ylabel('$\gamma_{2}$')
            axs[1, 1].plot(point_x, point_y, marker='*', markersize=25, color='k')
            axs[2, 0].set_visible(False)
            axs[2, 1].set_visible(False)


# Add colorbar
cbar = fig.colorbar(sm,orientation='horizontal')
cbar.ax.set_position([0.19,0.15,0.65,0.15])
cbar.set_label('$- \ln \mathcal{L}(F)$')

plt.show()
fig.savefig('bpl_int.pdf')  



#________________ DIFFERENT NOISE LEVEL ___________________________



matplotlib.rc('xtick', labelsize=25) 
matplotlib.rc('ytick', labelsize=25) 
plt.rc('font', size=20)



wbk=np.loadtxt('bpl_dif_noise.dat')[:,0]
g1=np.loadtxt('bpl_dif_noise.dat')[:,1]
g2=np.loadtxt('bpl_dif_noise.dat')[:,5]
g3=np.loadtxt('bpl_dif_noise.dat')[:,3]
l1=np.loadtxt('bpl_dif_noise.dat')[:,2]
l2=np.loadtxt('bpl_dif_noise.dat')[:,6]
l3=np.loadtxt('bpl_dif_noise.dat')[:,4]


w_max, w_min, w_step = 0.7, 0.1, 0.001
ww = np.round(np.arange(w_min, w_max, w_step), 3) 

lf1, lf2, lf3, gg1, gg2, gg3 = [np.zeros(len(ww)) for i in range(6)]

for i, ww_ in enumerate(ww):
    lf1[i], lf2[i], lf3[i] = [np.mean(l[wbk == ww_]) for l in [l1, l2, l3]]
    gg1[i], gg2[i], gg3[i] = [np.mean(g[wbk == ww_]) for g in [g1, g2, g3]]


length = len(ww)
brk_indices = np.arange(0, len(wbk), length)
wbk_reshape = wbk.reshape(-1, length)
l_reshape = [l.reshape(-1, length) for l in [l1, l2, l3]]
min_indices = [np.argmin(l, axis=1) for l in l_reshape]
selected_indices = [np.random.choice(min_idx, size=len(brk_idx)) for min_idx, brk_idx in zip(min_indices, [brk_indices]*3)]
brk = [wbk_rsh[np.arange(len(brk_idx)), sel_idx] for wbk_rsh, sel_idx, brk_idx in zip([wbk_reshape]*3, selected_indices, [brk_indices]*3)]
brk1, brk2, brk3 = brk



g_indices = np.arange(0, len(wbk), length)
g_reshape = [g.reshape(-1,length) for g in [g1,g2,g3]]
gamma=[g_rsh[np.arange(len(brk_idx)), sel_idx] for g_rsh, sel_idx, brk_idx in zip(g_reshape, selected_indices, [g_indices]*3)]



gamma1, gamma2, gamma3 = gamma


# Define confidence levels
df = 3
s = 0.5 * chi2.ppf(0.68, df)
s2 = 0.5 * chi2.ppf(0.95, df)
s3 = 0.5 * chi2.ppf(0.997, df)

Level=[s,s2,s3]

# Calculate mean and covariance matrices
data_list = [np.column_stack((brk1, gamma1)), np.column_stack((brk2, gamma2)), 
             np.column_stack((brk3, gamma3))]
mean_list = [np.mean(data, axis=0) for data in data_list]
cov_list = [np.cov(data, rowvar=False) for data in data_list]

point_x = 0.3
point_y = 3.

# Create colorbar
cmap = plt.get_cmap('Set1')
fig, axs = plt.subplots(2,2, figsize=(20, 20))
for i, (lf, cov, mean, gg) in enumerate(zip([lf1, lf2, lf3], [cov_list[0], cov_list[1], cov_list[2]], mean_list, [gg1, gg2, gg3])):
    # Create figure and axis


    # Create colorbar
    vmin = lf.min()
    vmax = lf.max()
    norm = mcolors.TwoSlopeNorm(vmax=vmax, vcenter=(vmax+vmin)//2, vmin=vmin)
    sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])

    for j, conf_level in enumerate(Level):
        lambdai_, v = np.linalg.eig(cov)
        lambdai_ = np.sqrt(lambdai_)
        ell1 = Ellipse(xy=mean, width=lambdai_[0]*np.sqrt(Level[0])*2,
                        height=lambdai_[1]*np.sqrt(Level[0])*2,
                        angle=np.rad2deg(np.arccos(v[0, 0])), fill=True, alpha=1., facecolor=cmap(0))
        ell2 = Ellipse(xy=mean, width=lambdai_[0]*np.sqrt(Level[1])*2,
                        height=lambdai_[1]*np.sqrt(Level[1])*2,
                        angle=np.rad2deg(np.arccos(v[0, 0])), fill=True, alpha=0.2, facecolor=cmap(1))
        ell3 = Ellipse(xy=mean, width=lambdai_[0]*np.sqrt(Level[2])*2,
                        height=lambdai_[1]*np.sqrt(Level[2])*2,
                        angle=np.rad2deg(np.arccos(v[0, 0])), fill=True, alpha=0.2, facecolor=cmap(9))
        # Add Ellipse artists to subplot
        if i == 0:
                
            axs[0, 0].add_artist(ell1)
            axs[0, 0].add_artist(ell2)
            axs[0, 0].add_artist(ell3)
            axs[0, 0].set_xlim(0.07, 0.5)
            axs[0, 0].set_ylim(1.5, 3.5)
            axs[0, 0].plot(point_x, point_y, marker='*', markersize=25, color='k')
            axs[0, 0].set_title(r'$\frac{1}{2}$ $\times$ True Noise Level')
            axs[0, 0].set_xlabel('$\omega_{bk}$ (rad/day)')
            axs[0, 0].set_ylabel('$\gamma_{2}$')

          
        elif i == 1:
            axs[0, 1].add_artist(ell1)
            axs[0, 1].add_artist(ell2)
            axs[0, 1].add_artist(ell3)
            axs[0, 1].set_xlim(0.1, 0.6)
            axs[0, 1].set_ylim(1., 9.5)
            axs[0, 1].set_xlabel('$\omega_{bk}$ (rad/day)')
            axs[0, 1].set_ylabel('$\gamma_{2}$')

            axs[0, 1].set_title(r'$\frac{3}{2}$ $\times$ True Noise Level')
            axs[0, 1].plot(point_x, point_y, marker='*', markersize=25, color='k')

        
        elif i == 2:
            axs[1, 0].add_artist(ell1)
            axs[1, 0].add_artist(ell2)
            axs[1, 0].add_artist(ell3)
            axs[1, 0].set_xlim(0.1, 0.57)
            axs[1, 0].set_ylim(2.0, 4.)
            axs[1, 0].set_title('True Noise Level')
            axs[1, 0].set_xlabel('$\omega_{bk}$ (rad/day)')
            axs[1, 0].set_ylabel('$\gamma_{2}$')
            axs[1, 0].plot(point_x, point_y, marker='*', markersize=25, color='k')
            axs[1, 0].set_position([0.35,0.1,0.35,0.28])
            axs[1, 1].set_visible(False)
            
            
  
        
cbar = fig.colorbar(sm)
cbar.ax.set_position([0.75,0.1,0.52,0.28])
cbar.set_label('$- \ln \mathcal{L}(F)$')
            
# Show plot
plt.show()
fig.savefig('bpl_dif_noise.pdf')  


#________________ LOG NOISE LEVEL ___________________________

matplotlib.rc('xtick', labelsize=18) 
matplotlib.rc('ytick', labelsize=25) 
plt.rc('font', size=40)


data = np.loadtxt('bpl_dif_noise.dat')

wbk=data[:,0]

g = []
l = []

for i in range(1, 11):
    g.append(data[:, 2 * i - 1])
    l.append(data[:, 2 * i])

g1, g2, g3, g4, g5, g6, g7, g8, g9, g10 = g
l1, l2, l3, l4, l5, l6, l7, l8, l9, l10 = l


w_max, w_min, w_step = 0.6, 0.1, 0.001
ww = np.round(np.arange(w_min, w_max, w_step), 3) 

lf1, lf2, lf3, lf4, lf5, lf6, lf7, lf8, lf9, lf10, gg1, gg2, gg3, gg4, gg5, gg6, gg7, gg8, gg9, gg10 = [np.zeros(len(ww)) for i in range(20)]

for i, ww_ in enumerate(ww):
    lf1[i], lf2[i], lf3[i], lf4[i], lf5[i], lf6[i], lf7[i], lf8[i], lf9[i], lf10[i] = [np.mean(l[wbk == ww_]) for l in [l1, l2, l3, l4, l5, l6, l7, l8, l9, l10]]
    gg1[i], gg2[i], gg3[i], gg4[i], gg5[i], gg6[i], gg7[i], gg8[i], gg9[i], gg10[i] = [np.mean(g[wbk == ww_]) for g in [g1, g2, g3, g4, g5, g6, g7, g8, g9, g10]]


length = len(ww)
brk_indices = np.arange(0, len(wbk), length)
wbk_reshape = wbk.reshape(-1, length)
l_reshape = [l.reshape(-1, length) for l in [l1, l2, l3, l4, l5, l6, l7, l8, l9, l10]]
min_indices = [np.argmin(l, axis=1) for l in l_reshape]
selected_indices = [np.random.choice(min_idx, size=len(brk_idx)) for min_idx, brk_idx in zip(min_indices, [brk_indices]*10)]
brk = [wbk_rsh[np.arange(len(brk_idx)), sel_idx] for wbk_rsh, sel_idx, brk_idx in zip([wbk_reshape]*10, selected_indices, [brk_indices]*10)]
brk1, brk2, brk3, brk4, brk5, brk6, brk7, brk8, brk9, brk10 = brk


g_indices = np.arange(0, len(wbk), length)
g_reshape = [g.reshape(-1,length) for g in [g1,g2,g3,g4,g5,g6,g7,g8,g9,g10]]
gamma=[g_rsh[np.arange(len(brk_idx)), sel_idx] for g_rsh, sel_idx, brk_idx in zip(g_reshape, selected_indices, [g_indices]*10)]



gamma1, gamma2, gamma3, gamma4, gamma5, gamma6, gamma7, gamma8, gamma9, gamma10 = gamma


w0=0.3 # true break frequency
slope=3.0 # true high frequency slope

x=np.array([0.5, 0.57009, 0.64565, 0.72654, 0.81283, 0.90485, 1.002, 1.1049, 1.2144, 2])
y=(1/w0)*np.mean(brk,axis=1)  # ratio of the fitted break frequency to the true break frequency
y_err=np.std(brk,axis=1) # error bars on the fitted break frequency
z=np.mean(gamma,axis=1)/slope # ratio of the fitted slope / true slope 
z_err=(1/slope)*np.std(gamma,axis=1) # error bars on the fitted slope

# Overplotting the results from a single simulation

# Convert the 'brk' and 'gamma' lists into NumPy arrays

brk = np.array(brk)
gamma = np.array(gamma)

# Calculate the mean of each row
row_means = np.mean(brk, axis=1)

# Calculate the differences between each element and the mean for each row
diffs = np.abs(brk - row_means[:, np.newaxis])

# Calculate the combined differences for each index
combined_diffs = np.sum(diffs, axis=0)

# Find the index closest to the mean for each row
closest_indices = np.abs(brk - row_means[:, np.newaxis]).argmin(axis=1)

# Use the closest_indices to extract the corresponding values from each row
single_sim_wbk = (1/w0) * brk[np.arange(brk.shape[0]), closest_indices]
single_sim_slp = (1/slope) * gamma[np.arange(brk.shape[0]), closest_indices]


# Create the subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 15))

# First subplot
ax1.plot(x, y, marker='o', color='blue', linewidth=4, label='Mean of 100 simulations')
ax1.fill_between(x, y-y_err, y+y_err, color='red', alpha=0.4,label=r'$1-\sigma$ deviation of 100 simulations')
ax1.plot(x, single_sim_wbk, marker='d', color='k', linewidth=4, label='Single Simulation')
ax1.set_xlabel(r'Relative noise level in units of $\sigma_{F}$')
ax1.set_xscale('log')
ax1.set_ylabel(r'$\frac{\omega_{(best-fit)}}{\omega_{true}}$')
ax1.grid(True)
ax1.legend()
ax1.set_xticks([0.5, 0.81283, 1.002,  1.2144, 2])
ax1.set_xticklabels([0.5, 0.81283, 1.002,  1.2144, 2])

ax1.legend(loc='upper left', fontsize=22)


# Second subplot
ax2.plot(x, z, marker='o', color='blue', linewidth=4)
ax2.fill_between(x, z-z_err, z+z_err, color='red', alpha=0.4)  # Add shaded error region
ax2.plot(x, single_sim_slp, marker='d', color='k', linewidth=4)
ax2.set_xlabel(r'Relative noise level in units of $\sigma_{F}$')
ax2.set_xscale('log')
ax2.set_ylabel(r'$\frac{\gamma_{2 (best-fit)} }{ \gamma_{2 (true)}}$')
ax2.grid(True)
ax2.set_xticks([0.5, 0.81283, 1.002,  1.2144, 2])
ax2.set_xticklabels([0.5, 0.81283, 1.002,  1.2144, 2])

# Adjust the spacing between the subplots
plt.subplots_adjust(wspace=0.2)

# Save and display the plot
plt.savefig("slope_omega_noise.pdf")
plt.show()

# <span style="font-family: 'Trebuchet MS', cursive, sans-serif; font-weight: bold; color: red;font-size: 48px;">For The Kepler Sample</span>

# <span style="font-family: 'Verdana', cursive, sans-serif; font-weight: bold; color: green;font-size: 40px;">For The Power Law PSD Model</span>

# <span style="font-family: 'Calibri', cursive, sans-serif; font-weight: bold; color: violet;font-size: 36px;">Time Domain Analysis</span>

# <span style="font-family: 'Palatino', cursive, sans-serif; font-weight: bold; color: blue;font-size: 30px;">Effect of Different Noise Level Prescriptions and Binning Timescales</span>

In [ ]:
cwd = os.getcwd() # get the current working directory

for filename in os.listdir(cwd):
    if filename.startswith('lcwerrors'):
        file_id = filename.split('_')[1].split('.')[0]
        kepler = np.loadtxt(filename)       
       
        t=np.round(kepler[:,0],4)
        lc=kepler[:,1]

        step=np.round(np.diff(t),4)
        
        bin_sizes=np.array([48])
        
        for bin_size in bin_sizes:
            
            points=len(lc)
            tolerance=90000.05
            cadence=np.min(step)


            # +1 is added so that the splitting is done after the last element which 
            # satisfies the condition. if not added, the last element will not be 
            # included in the range. 
    
            seq = np.split(t, np.where(step > tolerance*np.min(step))[0] +1)
    
            l = []
    
            for s in seq:
                if len(s) >= points:
                    l.append((s[0], s[-1]))
                    
            for x in range (len(l)):
                    
            # section to find the starting and the end indices of the time segments
                    
                def t_ini(x):
                
                    starting_index=int(np.where(t==np.min(l[x]))[0])
                
                    return starting_index
    
                def t_fin(x):
                
                    ending_index=int(np.where(t==np.max(l[x]))[0])
                
                    return ending_index
    
    
    
                t_i=t_ini(x)
                t_f=t_fin(x)    
                
                
                f=interpolate.interp1d(t[t_i:t_f+1],lc[t_i:t_f+1],kind='linear')
                       
                # # evenly spaced time series segments
                  
                T=np.arange(t[t_i],t[t_f],cadence)
    
                # new flux after interpolation
    
                Ft_=f(T)
            
               
                length=len(Ft_)
                   
                index=int(points//bin_size)*bin_size # index upto which Ft1 is divisible by binning size
                   
                Ft1=Ft_[:index]
           
    
                Ft=np.mean(Ft1.reshape(-1,bin_size),axis=1)
            
                N=len(Ft)
                 
                w=2*(np.pi)*np.linspace(1,N,N)/N/cadence
                
                def power (gamma):
                    p = w**(-gamma)
                    return p
                
            # Noise level prescriptions
            
                sigma_F=np.abs(np.mean(np.abs((np.diff(Ft))))/np.sqrt(bin_size))  # No 1
                
          #      sigma_F=np.abs(np.mean(np.abs((np.sqrt(Ft))))/np.sqrt(bin_size))  # No 2
                
          #      sigma_F = np.abs(np.mean(np.abs(kepler[:,2])))                     # No 3
                
          #      sigma_F = 1.1*(np.abs(np.mean(np.abs((np.diff(Ft))))/np.sqrt(bin_size))) 
    
                
                def Sigma_phi(alpha, gamma):
                    Sigma_phi_ = (alpha**2) * power(gamma)
                    return Sigma_phi_
                
                
                def cov_mat(alpha, gamma):
                    
                    dt = np.linspace(0, N*bin_size*cadence, N)
                    cov = Sigma_phi(alpha, gamma)
    
        # calculate sigma for all dt values
                    sigma = 2 * np.trapz(cov * np.cos(w * dt[:, np.newaxis]), w, axis=1)
    
        # add noise to the first element of sigma
                    sigma[0] += sigma_F**2
    
        # create the covariance matrix by subtracting the absolute difference between indices
                    ID = np.arange(sigma.size)
                    sigma = sigma[np.abs(ID - ID[:, None])]
    
                    return sigma
                
                def log_like_F (alpha,gamma):
            
                    mu=np.mean(Ft)
              
                    data=Ft-mu
            
                    inv=np.linalg.inv(cov_mat(alpha,gamma))
            
                    sign,logdet=np.linalg.slogdet(2*np.pi*cov_mat(alpha,gamma))
            
                    lh=-0.5*( logdet + np.transpose(data) @ inv @ data )
            
                    return lh
                
                
                def mnz(args):
                    alpha, gamma = args
                    return -log_like_F(alpha, gamma) + log_like_F(1000, 1)
    
    
                g1 = np.arange(1., 4.5, 0.1)
    
                diffe = np.zeros(len(g1))
                amp = np.zeros(len(g1))
    
                results = Parallel(n_jobs=multiprocessing.cpu_count())(
                  delayed(minimize)(mnz, (1, g1_), bounds=((0.000001, 100000), (g1_, g1_)))
                  for g1_ in g1
                )
    
                for j, res1 in enumerate(results):
                    amp[j] = res1.x[0]
                    diffe[j] = -log_like_F(amp[j], g1[j]) + log_like_F(1000, 1)
    
    
                rows=zip(g1,amp,diffe)
                
                
                with open(f'diff_bin_{bin_size}_{file_id}.dat', 'a') as f:
    #            with open(f'count_bin_{bin_size}_{file_id}.dat', 'a') as f:
    #            with open(f'smith_bin_{bin_size}_{file_id}.dat', 'a') as f:
                    writer = csv.writer(f, delimiter=' ')
                    for i in rows:
                        writer.writerow(i)

# <span style="font-family: 'Palatino', cursive, sans-serif; font-weight: bold; color: blue;font-size: 30px;">Effect of Incorrect Noise Level Assumptions</span>

In [ ]:
cwd = os.getcwd() # get the current working directory

for filename in os.listdir(cwd):
    if filename.startswith('lcwerrors'):
        file_id = filename.split('_')[1].split('.')[0]
        kepler = np.loadtxt(filename)       
       
        t=np.round(kepler[:,0],4)
        lc=kepler[:,1]

        step=np.round(np.diff(t),4)
        
        bin_sizes=np.array([1])
        
        for bin_size in bin_sizes:
            
            
            points=1000
            tolerance=3.05 
            cadence=np.min(step)


            # +1 is added so that the splitting is done after the last element which 
            # satisfies the condition. if not added, the last element will not be 
            # included in the range. 
    
            seq = np.split(t, np.where(step > tolerance*np.min(step))[0] +1)
    
            l = []
    
            for s in seq:
                if len(s) > points:
                    l.append((np.min(s), np.max(s)))
                    
            for x in range (len(l)):
                    
            # section to find the starting and the end indices of the time segments
                    
                def t_ini(x):
                
                    starting_index=int(np.where(t==np.min(l[x]))[0])
                
                    return starting_index
    
                def t_fin(x):
                
                    ending_index=int(np.where(t==np.max(l[x]))[0])
                
                    return ending_index
    
    
    
                t_i=t_ini(x)
                t_f=t_fin(x)    
                
                
                f=interpolate.interp1d(t[t_i:t_f+1],lc[t_i:t_f+1],kind='linear')
                       
                # # evenly spaced time series segments
                  
                T=np.arange(t[t_i],t[t_f],cadence)
    
                # new flux after interpolation
    
                Ft_=f(T)
            
               
                length=len(Ft_)
                   
                index=bin_size*(length//bin_size) # index upto which Ft1 is divisible by binning size
                   
                Ft1=Ft_[:index]
           
    
                Ft=np.mean(Ft1.reshape(-1,bin_size),axis=1)
            
                N=len(Ft)
                 
                w=2*(np.pi)*np.linspace(1,N,N)/N/cadence
                
                def power (gamma):
                    p = w**(-gamma)
                    return p
                
            # Noise level prescriptions
            
          #      sigma_F=np.abs(np.mean(np.abs((np.diff(Ft))))/np.sqrt(bin_size))  # No 1
                
          #      sigma_F=np.abs(np.mean(np.abs((np.sqrt(Ft))))/np.sqrt(bin_size))  # No 2
                
                sigma_F = np.abs(np.mean(np.abs(kepler[:,2])))                     # No 3
    
                
                def Sigma_phi(alpha, gamma):
                    Sigma_phi_ = (alpha**2) * power(gamma)
                    return Sigma_phi_
                
                
                def cov_mat(alpha, gamma, sigma_F):
                    
                    dt = np.linspace(0, N*bin_size*cadence, N)
                    cov = Sigma_phi(alpha, gamma)
    
        # calculate sigma for all dt values
                    sigma = 2 * np.trapz(cov * np.cos(w * dt[:, np.newaxis]), w, axis=1)
    
        # add noise to the first element of sigma
                    sigma[0] += sigma_F**2
    
        # create the covariance matrix by subtracting the absolute difference between indices
                    ID = np.arange(sigma.size)
                    sigma = sigma[np.abs(ID - ID[:, None])]
    
                    return sigma
                
                def log_like_F (alpha,gamma,sigma_F):
            
                    mu=np.mean(Ft)
              
                    data=Ft-mu
            
                    inv=np.linalg.inv(cov_mat(alpha,gamma,sigma_F))
            
                    sign,logdet=np.linalg.slogdet(2*np.pi*cov_mat(alpha,gamma,sigma_F))
            
                    lh=-0.5*( logdet + np.transpose(data) @ inv @ data )
            
                    return lh
                
                
                def mnz(args):
                    alpha, gamma, sigma_F = args
                    return -log_like_F(alpha, gamma, sigma_F) + log_like_F(10, 1, sigma_F)
    
    
                g1 = np.arange(1., 4.5, 0.1)
    
                # Noise level from 0.5 to 2.0 in 10 log steps
                
                ss1, ss2, ss3, ss4, ss5, ss6, ss7, ss8, ss9, ss10 = np.array([0.5, 0.57009, 0.64565, 0.72654, 0.81283, 0.90485, 1.002, 1.1049, 1.2144, 2]) * sigma_F
                diffe1, diffe2, diffe3, diffe4, diffe5, diffe6, diffe7, diffe8, diffe9, diffe10  = [np.zeros(len(g1)) for i in range(10)]
                amp1, amp2, amp3, amp4, amp5, amp6, amp7, amp8, amp9, amp10 = [np.zeros(len(g1)) for i in range(10)]
              
                
              
                results1 = Parallel(n_jobs=multiprocessing.cpu_count())(
                    delayed(minimize)(mnz, (1, g1_,ss1), bounds=((0.1, 10), (g1_, g1_),(ss1,ss1)))
                    for g1_ in g1
                )

                for j, res1 in enumerate(results1):
                    amp1[j] = res1.x[0]
                    diffe1[j] = -log_like_F(amp1[j], g1[j],ss1) + log_like_F(1, 1,ss1)
                    
                    
                results2 = Parallel(n_jobs=multiprocessing.cpu_count())(
                    delayed(minimize)(mnz, (1, g1_,ss2), bounds=((0.1, 10), (g1_, g1_),(ss2,ss2)))
                    for g1_ in g1
                )

                for j, res2 in enumerate(results2):
                    amp2[j] = res2.x[0]
                    diffe2[j] = -log_like_F(amp2[j], g1[j],ss2) + log_like_F(1, 1,ss2)
                    
              
                results3 = Parallel(n_jobs=multiprocessing.cpu_count())(
                    delayed(minimize)(mnz, (1, g1_,ss3), bounds=((0.1, 10), (g1_, g1_),(ss3,ss3)))
                    for g1_ in g1
                )

                for j, res3 in enumerate(results3):
                    amp3[j] = res3.x[0]
                    diffe3[j] = -log_like_F(amp3[j], g1[j],ss3) + log_like_F(1, 1,ss3)
                    
                    
                results4 = Parallel(n_jobs=multiprocessing.cpu_count())(
                    delayed(minimize)(mnz, (1, g1_,ss4), bounds=((0.1, 10), (g1_, g1_),(ss4,ss4)))
                    for g1_ in g1
                )

                for j, res4 in enumerate(results4):
                    amp4[j] = res4.x[0]
                    diffe4[j] = -log_like_F(amp4[j], g1[j],ss4) + log_like_F(1, 1,ss4)
                    
                    
                results5 = Parallel(n_jobs=multiprocessing.cpu_count())(
                    delayed(minimize)(mnz, (1, g1_,ss5), bounds=((0.1, 10), (g1_, g1_),(ss5,ss5)))
                    for g1_ in g1
                )

                for j, res5 in enumerate(results5):
                    amp5[j] = res5.x[0]
                    diffe5[j] = -log_like_F(amp5[j], g1[j],ss5) + log_like_F(1, 1,ss5)
                    
                    
                results6 = Parallel(n_jobs=multiprocessing.cpu_count())(
                    delayed(minimize)(mnz, (1, g1_,ss6), bounds=((0.1, 10), (g1_, g1_),(ss6,ss6)))
                    for g1_ in g1
                )

                for j, res6 in enumerate(results6):
                    amp6[j] = res6.x[0]
                    diffe6[j] = -log_like_F(amp6[j], g1[j],ss6) + log_like_F(1, 1,ss6)
                    
              
                results7 = Parallel(n_jobs=multiprocessing.cpu_count())(
                    delayed(minimize)(mnz, (1, g1_,ss7), bounds=((0.1, 10), (g1_, g1_),(ss7,ss7)))
                    for g1_ in g1
                )

                for j, res7 in enumerate(results7):
                    amp7[j] = res7.x[0]
                    diffe7[j] = -log_like_F(amp7[j], g1[j],ss7) + log_like_F(1, 1,ss7)
                    
                    
                results8 = Parallel(n_jobs=multiprocessing.cpu_count())(
                    delayed(minimize)(mnz, (1, g1_,ss8), bounds=((0.1, 10), (g1_, g1_),(ss8,ss8)))
                    for g1_ in g1
                )

                for j, res8 in enumerate(results8):
                    amp8[j] = res8.x[0]
                    diffe8[j] = -log_like_F(amp8[j], g1[j],ss8) + log_like_F(1, 1,ss8)
                    
                    
                results9 = Parallel(n_jobs=multiprocessing.cpu_count())(
                    delayed(minimize)(mnz, (1, g1_,ss8), bounds=((0.1, 10), (g1_, g1_),(ss9,ss9)))
                    for g1_ in g1
                )

                for j, res9 in enumerate(results9):
                    amp9[j] = res9.x[0]
                    diffe9[j] = -log_like_F(amp9[j], g1[j],ss9) + log_like_F(1, 1,ss9)
                    
                    
                results10 = Parallel(n_jobs=multiprocessing.cpu_count())(
                    delayed(minimize)(mnz, (1, g1_,ss10), bounds=((0.1, 10), (g1_, g1_),(ss10,ss10)))
                    for g1_ in g1
                )

                for j, res10 in enumerate(results10):
                    amp10[j] = res10.x[0]
                    diffe10[j] = -log_like_F(amp10[j], g1[j],ss10) + log_like_F(1, 1,ss10)
                    
                    
                    
                rows=zip(np.round(g1,2),np.round(diffe1,2),np.round(diffe2,2),np.round(diffe3,2),np.round(diffe4,2),
                         np.round(diffe5,2),np.round(diffe6,2),np.round(diffe7,2),np.round(diffe8,2),
                         np.round(diffe9,2),np.round(diffe10,2))
                
                
                with open(f'log_noise3_{bin_size}_{file_id}.dat', 'a') as f:
                    writer = csv.writer(f, delimiter=' ')
                    for i in rows:
                        writer.writerow(i)

# <span style="font-family: 'Century Gothic', cursive, sans-serif; font-weight: bold; color: purple;font-size: 30px;">Template for Plots</span>

In [ ]:
#_____________ For Different Noise Level _______________________ 

#__________________ For Noise Level Prescription 1 __________________________

file_id_list = []  # to store the file_id
slp_dict = {}
std_dict = {}
bin_sizes = [1, 4, 16, 64]

cwd = os.getcwd() # get the current working directory

for size in bin_sizes:
    slp_dict[size] = []
    std_dict[size] = []
    prefix = f"diff_bin_{size}_"
    
    for filename in os.listdir(cwd):
        if filename.startswith(prefix):
            file_id = filename.split('_')[3].split('.')[0]
            file_id_list.append(file_id)
            kepler = np.loadtxt(filename)       
            slope=kepler[:,0]
            ll=kepler[:,2]
            slp=np.arange(1., 4.5, 0.1)
            llf = np.zeros(len(slope))
            length=len(slp)
            for i in range(0, len(slope), length):
                min_indices = np.where(ll[i:i+length] == np.min(ll[i:i+length]))[0]
                random_index = np.random.choice(min_indices)
                llf[i] = slope[i:i+length][random_index]
                
            llf = llf[llf != 0]
            slp_dict[size].append(round(np.mean(llf), 2))
            std_dict[size].append(round(np.std(llf), 2))


file_id_list=np.unique(file_id_list)


#__________________ For Noise Level Prescription 2 __________________________



file_id_list = []  # to store the file_id
slp_dict2 = {}
std_dict2 = {}
bin_sizes = [1, 4, 16, 64]

cwd = os.getcwd() # get the current working directory

for size in bin_sizes:
    slp_dict2[size] = []
    std_dict2[size] = []
    prefix = f"count_bin_{size}_"
    
    for filename in os.listdir(cwd):
        if filename.startswith(prefix):
            file_id = filename.split('_')[3].split('.')[0]
            file_id_list.append(file_id)
            kepler = np.loadtxt(filename)       
            slope=kepler[:,0]
            ll=kepler[:,2]
            slp=np.arange(1., 4.5, 0.1)
            llf = np.zeros(len(slope))
            length=len(slp)
            for i in range(0, len(slope), length):
                min_indices = np.where(ll[i:i+length] == np.min(ll[i:i+length]))[0]
                random_index = np.random.choice(min_indices)
                llf[i] = slope[i:i+length][random_index]
    
            llf = llf[llf != 0]
            slp_dict2[size].append(round(np.mean(llf), 2))
            std_dict2[size].append(round(np.std(llf), 2))


file_id_list=np.unique(file_id_list)


#__________________ For Noise Level Prescription 3 __________________________



file_id_list = []  # to store the file_id
slp_dict3 = {}
std_dict3 = {}
bin_sizes = [1, 4, 16, 64]

cwd = os.getcwd() # get the current working directory

for size in bin_sizes:
    slp_dict3[size] = []
    std_dict3[size] = []
    prefix = f"smith_bin_{size}_"
    
    for filename in os.listdir(cwd):
        if filename.startswith(prefix):
            file_id = filename.split('_')[3].split('.')[0]
            file_id_list.append(file_id)
            
            kepler = np.loadtxt(filename)       
            slope=kepler[:,0]
            ll=kepler[:,2]
            slp=np.arange(1., 4.5, 0.1)
            llf = np.zeros(len(slope))
            length=len(slp)
            for i in range(0, len(slope), length):
                min_indices = np.where(ll[i:i+length] == np.min(ll[i:i+length]))[0]
                random_index = np.random.choice(min_indices)
                llf[i] = slope[i:i+length][random_index]
    
            llf = llf[llf != 0]
            slp_dict3[size].append(round(np.mean(llf), 2))
            std_dict3[size].append(round(np.std(llf), 2))


file_id_list=np.unique(file_id_list)


#______________________________Plot___________________________________________

# Set the font size for tick labels
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)
plt.rc('font', size=30)

fig, axes = plt.subplots(1, 3, figsize=(35, 12))  # Increase the figsize to make the plots bigger
plt.subplots_adjust(wspace=0.5, hspace=0.5)

markers = ['o', 's', '^', '*', 'v', 'p', 'D', 'X', 'd', '<', '>', '1', '2', '3', '4', '8', 'h', 'H', '+', 'x', '|']
bin_time = np.array(bin_sizes) * 0.5

legend_labels = []  # List to store legend labels

for i in range(len(slp_dict[1])):
    axes[0].plot(bin_time, [slp_dict[1][i], slp_dict[4][i], slp_dict[16][i], slp_dict[64][i]], marker=markers[i], label=file_id_list[i])
    axes[1].plot(bin_time, [slp_dict2[1][i], slp_dict2[4][i], slp_dict2[16][i], slp_dict2[64][i]], marker=markers[i], label=file_id_list[i])
    axes[2].plot(bin_time, [slp_dict3[1][i], slp_dict3[4][i], slp_dict3[16][i], slp_dict3[64][i]], marker=markers[i], label=file_id_list[i])

    legend_labels.append(file_id_list[i])  # Add legend label to the list

axes[0].set_xlabel('Bin timescale (Hours)')
axes[0].set_ylabel('PSD Slope')
axes[0].set_xscale('log')
axes[0].set_ylim(1.0, 4.5)
axes[0].set_xticks(bin_time)
axes[0].set_xticklabels([bin_time[0], bin_time[1], bin_time[2], bin_time[3]])

axes[1].set_xlabel('Bin timescale (Hours)')
axes[1].set_xscale('log')
axes[1].set_ylim(1.0, 4.5)
axes[1].set_yticks([])
axes[1].set_xticks(bin_time)
axes[1].set_xticklabels([bin_time[0], bin_time[1], bin_time[2], bin_time[3]])

axes[2].set_xlabel('Bin timescale (Hours)')
axes[2].set_xscale('log')
axes[2].set_ylim(1.0, 4.5)
axes[2].set_yticks([])
axes[2].set_xticks(bin_time)
axes[2].set_xticklabels([bin_time[0], bin_time[1], bin_time[2], bin_time[3]])

legend = axes[2].legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=16)
legend.set_title("Kepler ID")

plt.tight_layout()
plt.savefig('slp_vs_bin.pdf')
plt.show()

#__________________ For Different Noise Level Relative to the True Level ____________



cwd = os.getcwd() # get the current working directory

for filename in os.listdir(cwd):
    if filename.startswith('lcwerrors'):
        file_id = filename.split('_')[1].split('.')[0]
        kepler = np.loadtxt(filename)       
       
        t=np.round(kepler[:,0],4)
        lc=kepler[:,1]

        step=np.round(np.diff(t),4)
        
        bin_sizes=np.array([1])
        
        for bin_size in bin_sizes:
            
            
            points=1000
            tolerance=3.05 
            cadence=np.min(step)


            # +1 is added so that the splitting is done after the last element which 
            # satisfies the condition. if not added, the last element will not be 
            # included in the range. 
    
            seq = np.split(t, np.where(step > tolerance*np.min(step))[0] +1)
    
            l = []
    
            for s in seq:
                if len(s) > points:
                    l.append((np.min(s), np.max(s)))
                    
            for x in range (len(l)):
                    
            # section to find the starting and the end indices of the time segments
                    
                def t_ini(x):
                
                    starting_index=int(np.where(t==np.min(l[x]))[0])
                
                    return starting_index
    
                def t_fin(x):
                
                    ending_index=int(np.where(t==np.max(l[x]))[0])
                
                    return ending_index
    
    
    
                t_i=t_ini(x)
                t_f=t_fin(x)    
                
                
                f=interpolate.interp1d(t[t_i:t_f+1],lc[t_i:t_f+1],kind='linear')
                       
                # # evenly spaced time series segments
                  
                T=np.arange(t[t_i],t[t_f],cadence)
    
                # new flux after interpolation
    
                Ft_=f(T)
            
               
                length=len(Ft_)
                   
                index=bin_size*(length//bin_size) # index upto which Ft1 is divisible by binning size
                   
                Ft1=Ft_[:index]
           
    
                Ft=np.mean(Ft1.reshape(-1,bin_size),axis=1)
        
        sigma_F=np.abs(np.mean(np.abs((np.diff(Ft)))))


        
#________________ For Noise Level 1 __________________________________

       
        
file_id_list = []  # to store the file_id
slp_dict1 = {}
noise_level = np.array([0.5, 0.57009, 0.64565, 0.72654, 0.81283, 0.90485, 1.002, 1.1049, 1.2144, 2]) * sigma_F

prefix = f"log_noise1_"

for filename in os.listdir(cwd):
    if filename.startswith(prefix):
        file_id = filename.split('_')[3].split('.')[0]
        file_id_list.append(file_id)
        
        kepler = np.loadtxt(filename)
        
        slope = kepler[:, 0]
        
        for col in range(1, 11):
            
            ll = kepler[:, col]
            slp = np.arange(1., 4.5, 0.1)
            llf = np.zeros(len(slope))
            length = len(slp)
            for i in range(0, len(slope), length):
                min_indices = np.where(ll[i:i+length] == np.min(ll[i:i+length]))[0]
                random_index = np.random.choice(min_indices)
                llf[i] = slope[i:i+length][random_index]

            llf = llf[llf != 0]
            
            noise = noise_level[col - 1]  # Get the noise level based on column index
            
            if noise not in slp_dict1:
                slp_dict1[noise] = []
                
            
            slp_dict1[noise].append(round(np.mean(llf), 2))      
        
file_id_list=np.unique(file_id_list)        
  


# _________________ For Noise level 2 __________________________________      
        
slp_dict2 = {}

prefix = f"log_noise2_"

for filename in os.listdir(cwd):
    if filename.startswith(prefix):
        
        kepler = np.loadtxt(filename)
        
        slope = kepler[:, 0]
        
        for col in range(1, 11):
            
            ll = kepler[:, col]
            slp = np.arange(1., 4.5, 0.1)
            llf = np.zeros(len(slope))
            length = len(slp)
            for i in range(0, len(slope), length):
                min_indices = np.where(ll[i:i+length] == np.min(ll[i:i+length]))[0]
                random_index = np.random.choice(min_indices)
                llf[i] = slope[i:i+length][random_index]

            llf = llf[llf != 0]
            
            noise = noise_level[col - 1]  # Get the noise level based on column index
            
            if noise not in slp_dict2:
                slp_dict2[noise] = []
                
            
            slp_dict2[noise].append(round(np.mean(llf), 2))      
        
        
# _________________ For Noise level 3 __________________________________      
        
slp_dict3 = {}

prefix = f"log_noise3_"

for filename in os.listdir(cwd):
    if filename.startswith(prefix):
        
        kepler = np.loadtxt(filename)
        
        slope = kepler[:, 0]
        
        for col in range(1, 11):
            
            ll = kepler[:, col]
            slp = np.arange(1., 4.5, 0.1)
            llf = np.zeros(len(slope))
            length = len(slp)
            for i in range(0, len(slope), length):
                min_indices = np.where(ll[i:i+length] == np.min(ll[i:i+length]))[0]
                random_index = np.random.choice(min_indices)
                llf[i] = slope[i:i+length][random_index]

            llf = llf[llf != 0]
            
            noise = noise_level[col - 1]  # Get the noise level based on column index
            
            if noise not in slp_dict3:
                slp_dict3[noise] = []
                
            
            slp_dict3[noise].append(round(np.mean(llf), 2))       
        
        
#________________ Plot ______________________________________


# Set the font size for tick labels
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)
plt.rc('font', size=30)

# Extract the noise levels and values from slp_dict1
noises = list(slp_dict1.keys())
values1 = list(slp_dict1.values())
values2 = list(slp_dict2.values())
values3 = list(slp_dict3.values())

fig, axes = plt.subplots(1, 3, figsize=(35, 12))  # Increase the figsize to make the plots bigger
plt.subplots_adjust(wspace=0.5, hspace=0.5)

markers = ['o', 's', '^', '*', 'v', 'p', 'D', 'X', 'd', '<', '>', '1', '2', '3', '4', '8', 'h', 'H', '+', 'x', '|']

legend_labels = []  # List to store legend labels

# Iterate over the columns and plot each column with the noise level
for i in range(len(values1[0])):
    column1 = [row[i] for row in values1]
    column2 = [row[i] for row in values2]
    column3 = [row[i] for row in values3]
    
    # Plot the column values with the noise level, assigning markers and labels
    axes[0].plot(noises/sigma_F, column1, marker=markers[i], label=file_id_list[i])
    axes[1].plot(noises/sigma_F, column2, marker=markers[i], label=file_id_list[i])
    axes[2].plot(noises/sigma_F, column3, marker=markers[i], label=file_id_list[i])
    
    # Add legend label to the list
    legend_labels.append(file_id_list[i])

# Set plot labels and title for the first subplot
axes[0].set_xlabel(r'Relative noise level in units of $\sigma_{F}$')
axes[1].set_xlabel(r'Relative noise level in units of $\sigma_{F}$')
axes[2].set_xlabel(r'Relative noise level in units of $\sigma_{F}$')

axes[0].set_ylabel('PSD Slope')

axes[0].set_ylim(1.0, 4.5)
axes[1].set_ylim(1.0, 4.5)
axes[2].set_ylim(1.0, 4.5)

axes[1].set_yticks([])
axes[2].set_yticks([])

legend = axes[2].legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=16)
legend.set_title("Kepler ID")

plt.tight_layout()
plt.savefig('slp_vs_noise.pdf')
plt.show()

# <span style="font-family: 'Calibri', cursive, sans-serif; font-weight: bold; color: violet;font-size: 36px;">Frequency Domain Analysis</span>

In [ ]:
cwd = os.getcwd() # get the current working directory

for filename in os.listdir(cwd):
    if filename.startswith('lcwerrors'):
        file_id = filename.split('_')[1].split('.')[0]
        kepler = np.loadtxt(filename)       
       
        t=np.round(kepler[:,0],4)
        lc=kepler[:,1]

        step=np.round(np.diff(t),4)
        
        bin_sizes=np.array([144])
        
        for bin_size in bin_sizes:
            
            
            points=len(lc)
            tolerance=90000.05 
            cadence=np.min(step)


            # +1 is added so that the splitting is done after the last element which 
            # satisfies the condition. if not added, the last element will not be 
            # included in the range. 
    
            seq = np.split(t, np.where(step > tolerance*np.min(step))[0] +1)
    
            l = []
    
            for s in seq:
                if len(s) >= points:
                    l.append((s[0], s[-1]))
                    
            for x in range (len(l)):
                    
            # section to find the starting and the end indices of the time segments
                    
                def t_ini(x):
                
                    starting_index=int(np.where(t==np.min(l[x]))[0])
                
                    return starting_index
    
                def t_fin(x):
                
                    ending_index=int(np.where(t==np.max(l[x]))[0])
                
                    return ending_index
    
    
    
                t_i=t_ini(x)
                t_f=t_fin(x)    
                
                
                f=interpolate.interp1d(t[t_i:t_f+1],lc[t_i:t_f+1],kind='linear')
                       
                # # evenly spaced time series segments
                  
                T=np.arange(t[t_i],t[t_f],cadence)
    
                # new flux after interpolation
    
                Ft_=f(T)
            
               
                length=len(Ft_)
                   
                index=int(points//bin_size)*bin_size # index upto which Ft1 is divisible by binning size
                   
                Ft1=Ft_[:index]
           
    
                Ft=np.mean(Ft1.reshape(-1,bin_size),axis=1)
                
                # Fourier transform of Ft
                
                F_w = np.fft.fft(Ft, norm='ortho')
            
                N=len(Ft)
                 
                w=2*(np.pi)*np.linspace(1,N,N)/N/cadence
                
                def power (gamma):
                    p = w**(-gamma)
                    return p
                
            # Noise level prescriptions
            
                sigma_F=np.abs(np.mean(np.abs((np.diff(Ft))))/np.sqrt(bin_size))  # No 1
                
                def Sigma_phi(alpha, gamma):
                    Sigma_phi_ = (alpha**2) * power(gamma)
                    return Sigma_phi_
                
                
                def log_like_F (alpha,gamma):
            
                    Sigma_phi_ = Sigma_phi(alpha, gamma)
                    
                    power_F_ = (np.abs(F_w)**2)
                    
                    lh = -0.5*(np.sum(power_F_ / Sigma_phi_) + np.sum(np.log(2*np.pi*Sigma_phi_)))
                    
                    return lh
                
                
                def mnz(args):
                    alpha, gamma = args
                    return -log_like_F(alpha, gamma) + log_like_F(2000, 1)
    
    
                g1 = np.arange(1., 4.5, 0.1)
    
                diffe = np.zeros(len(g1))
                amp = np.zeros(len(g1))
    
                results = Parallel(n_jobs=multiprocessing.cpu_count())(
                  delayed(minimize)(mnz, (1000, g1_), bounds=((100, 1000000), (g1_, g1_)))
                  for g1_ in g1
                )
    
                for j, res1 in enumerate(results):
                    amp[j] = res1.x[0]
                    diffe[j] = -log_like_F(amp[j], g1[j]) + log_like_F(2000, 1)
    
    
                rows=zip(g1,amp,diffe)
                
                
                with open(f'freq_diff_bin_{bin_size}_{file_id}.dat', 'a') as f:
                    writer = csv.writer(f, delimiter=' ')
                    for i in rows:
                        writer.writerow(i)
                

# <span style="font-family: 'Century Gothic', cursive, sans-serif; font-weight: bold; color: purple;font-size: 30px;">Template for Plots</span>

In [ ]:
#__________________ For Files in Time domain __________________________

file_id_list = []  # to store the file_id
slp_dict = {}
std_dict = {}
bin_sizes = [1,12,48]

cwd = os.getcwd() # get the current working directory

for size in bin_sizes:
    slp_dict[size] = []
    std_dict[size] = []
    prefix = f"diff_bin_{size}_"
    
    for filename in os.listdir(cwd):
        if filename.startswith(prefix):
            file_id = filename.split('_')[3].split('.')[0]
            file_id_list.append(file_id)
            kepler = np.loadtxt(filename)       
            slope=kepler[:,0]
            ll=kepler[:,2]
            slp=np.arange(1., 4.5, 0.1)
            llf = np.zeros(len(slope))
            length=len(slp)
            for i in range(0, len(slope), length):
                min_indices = np.where(ll[i:i+length] == np.min(ll[i:i+length]))[0]
                random_index = np.random.choice(min_indices)
                llf[i] = slope[i:i+length][random_index]
                
            llf = llf[llf != 0]
            slp_dict[size].append(round(np.mean(llf), 2))
            std_dict[size].append(round(np.std(llf), 2))


file_id_list=np.unique(file_id_list)

#__________________ For Files in Frequency domain __________________________

file_id_list = []  # to store the file_id
slp_dict2 = {}

for size in bin_sizes:
    slp_dict2[size] = []
    prefix = f"freq_diff_bin_{size}_"
    
    for filename in os.listdir(cwd):
        if filename.startswith(prefix):
            file_id = filename.split('_')[4].split('.')[0]
            file_id_list.append(file_id)
            kepler = np.loadtxt(filename)       
            slope=kepler[:,0]
            ll=kepler[:,2]
            slp=np.arange(1., 4.5, 0.1)
            llf = np.zeros(len(slope))
            length=len(slp)
            for i in range(0, len(slope), length):
                min_indices = np.where(ll[i:i+length] == np.min(ll[i:i+length]))[0]
                random_index = np.random.choice(min_indices)
                llf[i] = slope[i:i+length][random_index]
                
            llf = llf[llf != 0]
            slp_dict2[size].append(round(np.mean(llf), 2))


file_id_list=np.unique(file_id_list)


# #______________________________Plot___________________________________________

# Set the font size for tick labels
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)
plt.rc('font', size=30)

fig, axes = plt.subplots(1, 3, figsize=(35, 12))  # Increase the figsize to make the plots bigger
plt.subplots_adjust(wspace=0.5, hspace=0.5)

markers = ['o', 's', '^', '*', 'v', 'p', 'D', 'X', 'd', '<', '>', '1', '2', '3', '4', '8', 'h', 'H', '+', 'x', '|']

legend_labels = []  # List to store legend labels

for i in range(len(slp_dict[1])):
    axes[0].scatter([slp_dict2[1][i]],[slp_dict[1][i]], marker=markers[i], label=file_id_list[i],linewidth=10)
    axes[1].scatter([slp_dict2[12][i]],[slp_dict[12][i]], marker=markers[i], label=file_id_list[i],linewidth=10)
    axes[2].scatter([slp_dict2[48][i]],[slp_dict[48][i]], marker=markers[i], label=file_id_list[i],linewidth=10)

    legend_labels.append(file_id_list[i])  # Add legend label to the list

axes[0].set_xlabel('PSD slope (frequency domain)')
axes[0].set_ylabel('PSD slope (time domain)')
axes[0].set_ylim(1., 3.)
axes[0].set_xlim(1., 3.)
axes[0].set_yticks(np.linspace(1.5, 3, 4))  # Set 5 yticks
axes[0].set_title('Bin timescale = 0.5 hours')


axes[1].set_xlabel('PSD slope (frequency domain)')
axes[1].set_ylim(1., 3.)
axes[1].set_xlim(1., 3.)
axes[1].set_yticks([])
axes[1].set_title('Bin timescale = 6 hours')

axes[2].set_xlabel('PSD slope (frequency domain)')
axes[2].set_ylim(1., 3.)
axes[2].set_xlim(1., 3.)
axes[2].set_yticks([])
axes[2].set_title('Bin timescale = 24 hours')

legend = axes[2].legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=16)
legend.set_title("Kepler ID")

# Calculating the r and p value

r1,p1=np.round(stats.pearsonr(slp_dict2[1],slp_dict[1]),2)
r2,p2=np.round(stats.pearsonr(slp_dict2[12],slp_dict[12]),2)
r3,p3=np.round(stats.pearsonr(slp_dict2[48],slp_dict[48]),2)

axes[0].text(1.7, 2.5, f'$r={r1}, p={p1}$')
axes[1].text(1.7, 2.5, f'$r={r2}, p={p2}$')
axes[2].text(1.7, 2.5, f'$r={r3}, p={p3}$')


plt.tight_layout()
plt.savefig('freq_vs_time_pl.pdf')
plt.show()

#_____________ Comparing with Smith (Power law case) ___________________________


# _______________ Time domain ____________________________


smith=np.array([2.4,2.4,2.1,2.8,2,2.7,3.3,1.9,2.8,2.5,2.5,2.4,2.2,3.4,2.5,2.3,2.5,2.4,1.7,3,2.9])


fig, axes = plt.subplots(1, 3, figsize=(35, 12))  # Increase the figsize to make the plots bigger
plt.subplots_adjust(wspace=0.5, hspace=0.5)

markers = ['o', 's', '^', '*', 'v', 'p', 'D', 'X', 'd', '<', '>', '1', '2', '3', '4', '8', 'h', 'H', '+', 'x', '|']

legend_labels = []  # List to store legend labels

for i in range(len(slp_dict[1])):
    axes[0].scatter([smith[i]],[slp_dict[1][i]], marker=markers[i], label=file_id_list[i],linewidth=10)
    axes[1].scatter([smith[i]],[slp_dict[12][i]], marker=markers[i], label=file_id_list[i],linewidth=10)
    axes[2].scatter([smith[i]],[slp_dict[48][i]], marker=markers[i], label=file_id_list[i],linewidth=10)

    legend_labels.append(file_id_list[i])  # Add legend label to the list

axes[0].set_xlabel('PSD slope (Smith 2018)')
axes[0].set_ylabel('PSD slope (time domain)')
axes[0].set_ylim(1., 3.)
axes[0].set_xlim(1., 3.5)
axes[0].set_yticks(np.linspace(1.5, 3., 4))  # Set 5 yticks
axes[0].set_title('Bin timescale = 0.5 hours')


axes[1].set_xlabel('PSD slope (Smith 2018)')
axes[1].set_ylim(1., 3.)
axes[1].set_xlim(1., 3.5)
axes[1].set_yticks([])
axes[1].set_title('Bin timescale = 6 hours')

axes[2].set_xlabel('PSD slope (Smith 2018)')
axes[2].set_ylim(1., 3.)
axes[2].set_xlim(1., 3.5)
axes[2].set_yticks([])
axes[2].set_title('Bin timescale = 24 hours')

legend = axes[2].legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=16)
legend.set_title("Kepler ID")

# Calculating the r and p value

r4,p4=np.round(stats.pearsonr(smith,slp_dict[1]),2)
r5,p5=np.round(stats.pearsonr(smith,slp_dict[12]),2)
r6,p6=np.round(stats.pearsonr(smith,slp_dict[48]),2)

axes[0].text(1.7, 2.5, f'$r={r4}, p={p4}$')
axes[1].text(1.7, 2.5, f'$r={r5}, p={p5}$')
axes[2].text(1.2, 2.3, f'$r={r6}, p={p6}$')


plt.tight_layout()
plt.savefig('time_vs_smith_pl.pdf')
plt.show()


# ____________ FREQUENCY DOMAIN ________________________


fig, axes = plt.subplots(1, 3, figsize=(35, 12))  # Increase the figsize to make the plots bigger
plt.subplots_adjust(wspace=0.5, hspace=0.5)

markers = ['o', 's', '^', '*', 'v', 'p', 'D', 'X', 'd', '<', '>', '1', '2', '3', '4', '8', 'h', 'H', '+', 'x', '|']

legend_labels = []  # List to store legend labels

for i in range(len(slp_dict[1])):
    axes[0].scatter([smith[i]],[slp_dict2[1][i]], marker=markers[i], label=file_id_list[i],linewidth=10)
    axes[1].scatter([smith[i]],[slp_dict2[12][i]], marker=markers[i], label=file_id_list[i],linewidth=10)
    axes[2].scatter([smith[i]],[slp_dict2[48][i]], marker=markers[i], label=file_id_list[i],linewidth=10)

    legend_labels.append(file_id_list[i])  # Add legend label to the list

axes[0].set_xlabel('PSD slope (Smith 2018)')
axes[0].set_ylabel('PSD slope (frequency domain)')
axes[0].set_ylim(1., 3.)
axes[0].set_xlim(1., 3.5)
axes[0].set_yticks(np.linspace(1.5, 3., 4))  # Set 5 yticks
axes[0].set_title('Bin timescale = 0.5 hours')


axes[1].set_xlabel('PSD slope (Smith 2018)')
axes[1].set_ylim(1., 3.)
axes[1].set_xlim(1., 3.5)
axes[1].set_yticks([])
axes[1].set_title('Bin timescale = 6 hours')

axes[2].set_xlabel('PSD slope (Smith 2018)')
axes[2].set_ylim(1., 3.)
axes[2].set_xlim(1., 3.5)
axes[2].set_yticks([])
axes[2].set_title('Bin timescale = 24 hours')

legend = axes[2].legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=18)
legend.set_title("Kepler ID")


# Calculating the r and p value

r7,p7=np.round(stats.pearsonr(smith,slp_dict2[1]),2)
r8,p8=np.round(stats.pearsonr(smith,slp_dict2[12]),2)
r9,p9=np.round(stats.pearsonr(smith,slp_dict2[48]),2)

axes[0].text(1.7, 2.5, f'$r={r7}, p={p7}$')
axes[1].text(1.7, 2.5, f'$r={r8}, p={p8}$')
axes[2].text(1.2, 2.5, f'$r={r9}, p={p9}$')


plt.tight_layout()
plt.savefig('freq_vs_smith_pl.pdf')
plt.show()




#___________ Plotting TIME & FREQ for 1 object ____________________

file_id_list = []  # to store the file_id
slp_dict = {}
ll_dict = {}
bin_sizes = [48]

cwd = os.getcwd()  # get the current working directory


for size in bin_sizes:
    slp_dict[size] = []
    ll_dict[size] = []
    prefix = f"diff_bin_{size}"

    for filename in os.listdir(cwd):
        if filename.startswith(prefix):
            file_id = filename.split('_')[3].split('.')[0]
            file_id_list.append(file_id)
            kepler = np.loadtxt(filename)
            slope = kepler[:, 0]
            ll = kepler[:, 2]
            slp_dict[size].append(slope)
            ll_dict[size].append(ll)

file_id_list = np.unique(file_id_list)


ll2_dict = {}

# Define the figure size
plt.figsize = (20,20)  # Adjust the values as needed
# Set the font size for tick labels
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)
plt.rc('font', size=30)


for size in bin_sizes:
    ll2_dict[size] = []
    prefix = f"freq_diff_bin_{size}"

    for filename in os.listdir(cwd):
        if filename.startswith(prefix):
            kepler = np.loadtxt(filename)
            ll2 = kepler[:, 2]
            ll2_dict[size].append(ll2)

# Create separate plots for each file_id
for file_id in file_id_list:
    plt.figure()  # Create a new figure for each file_id
    for size in bin_sizes:
        for i in range(len(slp_dict[size])):
            if file_id in file_id_list[i]:
                plt.figure(figsize=(20, 20))
                plt.title(f'Kepler ID: {file_id}')
                plt.plot(slp_dict[size][i], ll_dict[size][i], linewidth=6,color='r',label='Time domain')
                plt.plot(slp_dict[size][i], ll2_dict[size][i], linewidth=6,color='b',label='Frequency domain')
                plt.ylim(1.8*np.min(ll2),100)
                plt.ylabel('$ - \ln \mathcal{L}(F)$')
                plt.xlabel(r'$\gamma$', fontsize=30.0)
                plt.legend(loc='center')
                
                # Generate a unique filename based on file_id
                filename = f'time_vs_freq_{file_id}_{size}_{i}.pdf'
                # Check if the filename starts with "578" before saving
                if file_id.startswith('578'):
                
                # Save the figure with the unique filename
                    plt.savefig(filename)
    
    plt.show()

# <span style="font-family: 'Verdana', cursive, sans-serif; font-weight: bold; color: blue;font-size: 40px;">For The Broken Power Law PSD Model</span>

In [ ]:
cwd = os.getcwd() # get the current working directory

for filename in os.listdir(cwd):
    if filename.startswith('lcwerrors'):
        file_id = filename.split('_')[1].split('.')[0]
        kepler = np.loadtxt(filename) 
        
       
        t=np.round(kepler[:,0],4)
        lc=kepler[:,1]

        step=np.round(np.diff(t),4)
        
        bin_sizes=np.array([1])
        
        
        for bin_size in bin_sizes:
            
            
            points=len(lc)
            tolerance=90000.05 
            cadence=np.min(step)


            # +1 is added so that the splitting is done after the last element which 
            # satisfies the condition. if not added, the last element will not be 
            # included in the range. 
    
            seq = np.split(t, np.where(step > tolerance*np.min(step))[0] +1)
    
            l = []
    
            for s in seq:
                if len(s) >= points:
                    l.append((np.min(s), s[-1]))
                    
            for x in range (len(l)):
                    
            # section to find the starting and the end indices of the time segments
                    
                def t_ini(x):
                
                    starting_index=int(np.where(t==np.min(l[x]))[0])
                
                    return starting_index
    
                def t_fin(x):
                
                    ending_index=int(np.where(t==np.max(l[x]))[0])
                
                    return ending_index
    
    
    
                t_i=t_ini(x)
                t_f=t_fin(x)    
                
                
                f=interpolate.interp1d(t[t_i:t_f+1],lc[t_i:t_f+1],kind='linear',fill_value='extrapolate')
                       
                # # evenly spaced time series segments
                  
                T=np.arange(t[t_i],t[t_f],cadence)
    
                # new flux after interpolation
    
                Ft_=f(T)
                   
                index=int(points//bin_size)*bin_size # index upto which Ft1 is divisible by binning size
                   
                Ft1=Ft_[:index]
           
    
                Ft=np.mean(Ft1.reshape(-1,bin_size),axis=1)
                
                F_w = np.fft.fft(Ft, norm='ortho')
            
                N=len(Ft)
                 
                w=2*(np.pi)*np.linspace(1,N,N)/N/cadence
                
                def power (gamma1,gamma2,w0):
                    
                  p=np.zeros(len(w))
                        
                  for i in range(len(w)):
                      if (w[i]<=w0):
                          p[i] = (w[i]/w0)**(-gamma1)
                      else:
                          p[i] = (w[i]/w0)**(-gamma2)
                                
                  return p
                
            # Noise level prescription
            
                sigma_F=np.abs(np.mean(np.abs((np.diff(Ft))))/np.sqrt(bin_size))   
                
                def Sigma_phi(alpha,gamma1,gamma2,w0):
                    Sigma_phi_ = (alpha**2) * power(gamma1,gamma2,w0)
                    return Sigma_phi_
                
                
                def log_like_F (alpha,gamma1,gamma2,w0):
            
                    Sigma_phi_ = Sigma_phi(alpha,gamma1,gamma2,w0)
                    
                    power_F_ = (np.abs(F_w)**2)
                    
                    lh = -0.5*(np.sum(power_F_ / Sigma_phi_) + np.sum(np.log(2*np.pi*Sigma_phi_)))
                    
                    return lh
                
                
                def mnz(args):
                    alpha, gamma1, gamma2, w0 = args
                    return -log_like_F(alpha,gamma1,gamma2,w0) + log_like_F(5000,1,1,1)
    
    
                # Frequency range 
                
                t_low, t_high =0.125, 100  # DAYS
                freq_step=1/(1*t[-1])
    
                wbk = np.arange(1/t_high,1/t_low,freq_step)
                
    
                diffe = np.zeros(len(wbk))
                amp = np.zeros(len(wbk))
                ind1 = np.zeros(len(wbk))
                ind2 = np.zeros(len(wbk))
    
                results = Parallel(n_jobs=multiprocessing.cpu_count())(
                  delayed(minimize)(mnz, (10000,1,2.5,wbk_), bounds=((100, 1000000),(1,1),(1,4),(wbk_, wbk_)))
                  for wbk_ in wbk
                )
    
                for j, res1 in enumerate(results):
                    amp[j] = res1.x[0]
                    ind1[j]=float(res1.x[1])
                    ind2[j]=float(res1.x[2])
                    diffe[j] = -log_like_F(amp[j],ind1[j],ind2[j],wbk[j]) + log_like_F(5000,1,1,1)
    
    
                rows=zip(wbk,amp,ind2,diffe)
                
                
                with open(f'freq_bpl_diff_bin_{bin_size}_{file_id}.dat', 'w') as f:
                    writer = csv.writer(f, delimiter=' ')
                    for i in rows:
                        writer.writerow(i)
                

# <span style="font-family: 'Century Gothic', cursive, sans-serif; font-weight: bold; color: purple;font-size: 30px;">Template for Plots</span>

In [ ]:
#____________ Obtaining the break freq and the slope _________________

file_id_list = []  # to store the file_id
slp_dict = {}
w_dict = {}
bin_sizes = [1]

cwd = os.getcwd()  # get the current working directory

for size in bin_sizes:
    slp_dict[size] = []
    w_dict[size] = []
    prefix = f"freq_bpl_diff_bin_{size}_"

    for filename in os.listdir(cwd):
        if filename.startswith(prefix):
            file_id = filename.split('_')[5].split('.')[0]
            kepler = np.loadtxt(filename)
            w = kepler[:, 0]
            slope = kepler[:, 2]
            ll = kepler[:, 3]
            llf, llf2 = np.zeros(len(w)), np.zeros(len(w))
            length = len(w)
            for i in range(0, len(w), length):
                min_indices = np.where(ll[i:i + length] == np.min(ll[i:i + length]))[0]
                random_index = np.random.choice(min_indices)
                if random_index > 0:

                    llf[i] = slope[random_index]
                    llf2[i] = w[random_index]


            llf = llf[llf != 0]
            if len(llf) > 0 and llf[i] > 1.0:  # Only append if there are valid values
                slp_dict[size].append(round(np.mean(llf), 2))
                w_dict[size].append(round(llf2[0], 4))
                file_id_list.append(file_id)
                
                
#__________ Correlation with BH mass, Lum, Edd ratio with the break timescales


# _______Black Hole Mass __________


bh = np.array([7.58, 7.52, 7.8, 7.38, np.nan, np.nan, 8.59, 
               7.9, 7.66, 8.49, 7.73, 7.43, np.nan, 7.3, 7.37])

bh_nan = np.isnan(bh)
w_bh = np.array(w_dict[1])[~bh_nan]
bh2 = bh[~bh_nan]
file_bh=np.array(file_id_list)[~bh_nan]

# _______ Luminosity _____________________

lum = np.array([44.77, 44.23, 44.32, 44.36, np.nan, np.nan, 45.78, 45.01, 44.71, 45.74, 
       44.18, 44.66, 44.95, 44.14, 44.4])

lum_nan = np.isnan(lum)
w_lum = np.array(w_dict[1])[~lum_nan]
lum2 = lum[~lum_nan]
file_lum=np.array(file_id_list)[~lum_nan]


#__________ Eddington ratio ________________


edd = np.array([0.124, 0.04, 0.026, 0.076, np.nan, np.nan, 0.124, 0.101, 0.089, 
       0.14, 0.023, 0.135, np.nan, 0.055, 0.085])

edd_nan = np.isnan(edd)
w_edd = np.array(w_dict[1])[~edd_nan]
edd2 = edd[~edd_nan]
file_edd=np.array(file_id_list)[~edd_nan]
                               
r1,p1=np.round(stats.pearsonr(bh2,1/w_bh),2)   
r2,p2=np.round(stats.pearsonr(lum2,1/w_lum),2)  
r3,p3=np.round(stats.pearsonr(edd2,1/w_edd),2)  

# Set the font size for tick labels
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)
plt.rc('font', size=30)


fig, axes = plt.subplots(1, 3, figsize=(35, 12))  # Increase the figsize to make the plots bigger
plt.subplots_adjust(wspace=0.2, hspace=0.5)


axes[0].scatter(bh2,1/w_bh, marker='D', color='r', linewidth=20)
axes[1].scatter(lum2,1/w_lum, marker='D', color='r', linewidth=20)
axes[2].scatter(edd2,1/w_edd, marker='D', color='r', linewidth=20)


axes[0].set_xlabel(r'$\log M_{\mathrm{BH}}\ (M_{\odot})$')
axes[0].set_ylabel(r'$T_{break}$ (Days)')

axes[1].set_xlabel(r'$\log L_{Bol}$ (erg $s^{-1}$)')
axes[1].set_yticks([])

axes[2].set_xlabel(r'$L/L_{Edd}$')
axes[2].set_yticks([])

# Setting 5 yticks for axes[0]

min_w_bh = int(np.min(1/w_bh))
max_w_bh = int(np.max(1/w_bh))

# Calculate the spacing between ticks
tick_spacing = (max_w_bh - min_w_bh) / 4  # Divide by 4 to get 5 ticks

# Generate a list of 5 ytick values evenly spaced between min and max
ytick_values = np.arange(min_w_bh, max_w_bh + tick_spacing, tick_spacing)

# Set the yticks for axes[0]
axes[0].set_yticks(ytick_values)


axes[0].text(7.5, 53, f'$r={r1}, p={p1}$')
axes[1].text(44.5, 53, f'$r={r2}, p={p2}$')
axes[2].text(0.05, 53, f'$r={r3}, p={p3}$')

plt.tight_layout()
plt.savefig('break_agn_prop.pdf')
plt.show()


# _______ Plotting log likelihood with break freq for a single object ___________


matplotlib.rc('xtick', labelsize=35) 
matplotlib.rc('ytick', labelsize=35) 
plt.rc('font', size=45)

for size in bin_sizes:
    slp_dict[size] = []
    w_dict[size] = []
    prefix = f"freq_bpl_diff_bin_{size}_12010193"

    for filename in os.listdir(cwd):
        if filename.startswith(prefix):
            file_id = filename.split('_')[5].split('.')[0]
            kepler = np.loadtxt(filename)
            w = kepler[:, 0]
            slope = kepler[:, 2]
            ll = kepler[:, 3]
            
            
            plt.figure(figsize=(25, 25))
            plt.plot(w, ll, linewidth=4, color='k')
            plt.title(f'Kepler ID: {file_id}')
            
            plt.ylabel('$ - \ln \mathcal{L}(F)$')
            plt.xlabel('$\omega_{bk}$ (rad/day)')
            plt.xscale('log')
            plt.savefig(f'bpl_{file_id}_wbk.pdf')
            
            plt.show()
            


#__________ Plotting the plane connecting M_BH, Lum and T_break ____________

file_id_list = []  # to store the file_id
slp_dict = {}
w_dict = {}
bin_sizes = [1]

cwd = os.getcwd()  # get the current working directory

for size in bin_sizes:
    slp_dict[size] = []
    w_dict[size] = []
    prefix = f"freq_bpl_diff_bin_{size}_"

    for filename in os.listdir(cwd):
        if filename.startswith(prefix):
            file_id = filename.split('_')[5].split('.')[0]
            kepler = np.loadtxt(filename)
            w = kepler[:, 0]
            slope = kepler[:, 2]
            ll = kepler[:, 3]
            llf, llf2 = np.zeros(len(w)), np.zeros(len(w))
            length = len(w)
            for i in range(0, len(w), length):
                min_indices = np.where(ll[i:i + length] == np.min(ll[i:i + length]))[0]
                random_index = np.random.choice(min_indices)
                if random_index > 0:

                    llf[i] = slope[random_index]
                    llf2[i] = w[random_index]


            llf = llf[llf != 0]
            if len(llf) > 0 and llf[i] > 1.0:  # Only append if there are valid values
                slp_dict[size].append(round(np.mean(llf), 2))
                w_dict[size].append(round(llf2[0], 4))
                file_id_list.append(file_id)
                    

matplotlib.rc('xtick', labelsize=35) 
matplotlib.rc('ytick', labelsize=35) 
plt.rc('font', size=45)          
                
bh = np.array([7.58, 7.52, 7.8, 7.38, np.nan, np.nan, 8.59, 
               7.9, 7.66, 8.49, 7.73, 7.43, np.nan, 7.3, 7.37])


lum = np.array([44.77, 44.23, 44.32, 44.36, np.nan, np.nan, 45.78, 45.01, 44.71, 45.74, 
       44.18, 44.66, np.nan, 44.14, 44.4])

bh_nan = np.isnan(bh)
lum_nan = np.isnan(lum)

x1= bh[~bh_nan]
y1= lum[~lum_nan]
z1 = 1/np.array(w_dict[1])[~lum_nan]

X_data = np.column_stack((x1,y1))
Y_data = z1

reg = linear_model.LinearRegression().fit(X_data, Y_data)

alpha1=reg.coef_[0]
alpha2=reg.coef_[1]
alpha3=reg.intercept_

alpha1=reg.coef_[0]
alpha2=reg.coef_[1]
alpha3=reg.intercept_

wbk=alpha1*x1[len(x1)-1]+alpha2*y1[len(y1)-1]+alpha3

matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 


fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(projection='3d')

ax.tick_params(axis='x', which='major', pad=-40)
ax.tick_params(axis='y', which='major', pad=-25)
ax.tick_params(axis='z', which='major', pad=-25)

ax.locator_params(axis='y', nbins=3)
ax.locator_params(axis='x', nbins=3)
ax.locator_params(axis='z', nbins=3)
  
ax.scatter(x1,y1,z1,marker='*',linewidth=25,c='r')
ax.set_xlabel(r'log $M_{BH}$ ($M_{\odot}$)',fontsize=35)
ax.set_ylabel(r'log $L_{Bol}$  (erg $s^{-1}$)',fontsize=35)
ax.set_zlabel('$T_{break}(Days)$',fontsize=35)
plt.setp(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.setp(ax.get_yticklabels(), rotation=45, horizontalalignment='right')
plt.tight_layout()


# Plot the best fit plane

xx, yy = np.meshgrid(x1, y1)
zz=alpha1*xx+alpha2*yy+alpha3
ax.plot_surface(xx, yy, zz, alpha=0.02,color='g')
#ax.text(6.7,44.5,60, r'$T = {alpha1} M_{BH} - {alpha2} L_{Bol} + {alpha3}$',fontsize=35)
# Format the text to display the variable values
text = r'$T = {:.2f} M_{{BH}} + {:.2f} L_{{Bol}}  {:.2f}$'.format(alpha1, alpha2, alpha3)
ax.text(7.5, 44.5, 80, text, color='b',fontsize=35)
plt.show()
fig.savefig('plane_break.pdf')